In [ ]:
import os
import numpy as np
import pandas as pd
import pytz
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy import optimize
import seaborn as sb
import pvlib as pv
from scipy.stats import gaussian_kde
import math

This notebook contains code used for generating the tables and figures in Khan et al. (2022). All figures, tables, and equations refered to in this notebook correspond with figures, tables, and equations in Khan et al. (2022).

Khan, A. M., Stoy, P. C., Joiner, J., Baldocchi, D., Verfaillie, J., Chen, M., & Otkin, J. A. (2022). The diurnal dynamics of Gross Primary Productivity using observations from the Advanced Baseline Imager on the Geostationary Operational Environmental Satellite-R Series at an oak savanna ecosystem. Journal of Geophysical Research: Biogeosciences, e2021JG006701. https://doi.org/10.1029/2021JG006701

# Read in data: Adjust file paths according to locations of files

In [ ]:
#Read in GOES values at tonzi
goes = pd.read_csv(r'path to file')

#Read in GOES 16 and 17 DSR values at tonzi 
goes_sw = pd.read_csv(r'path to file')

#Read in Ameriflux tonzi tower data
tonzi = pd.read_csv(r'path to file',
                    skiprows = 2
                   )

#Read in tonzi tower spectral dataset
tonzi_spec = pd.read_csv(r'path to file')

In [ ]:
#Mark -9999 as NA in ameriflux data
tonzi = tonzi.replace(-9999.0, np.NaN).copy()

# Fix all timestamps

In [ ]:
#Set UTC offset
offset = pytz.FixedOffset(-8 * 60)
amf_offset = '-0800'

In [ ]:
#Convert goes time to pandas datetime
goes.loc [:,'stimestamp_x'] = pd.to_datetime (goes.loc [:, 'stimestamp_x'])
goes.loc [:,'etimestamp_x'] = pd.to_datetime (goes.loc [:, 'etimestamp_x'])

#Make a new column which will turn into the index and then be converted to tonzi time
goes.loc[:, 'etimestamp_local_st'] = goes.loc[:, 'etimestamp_x']

#Set index as local standard time
goes.set_index('etimestamp_local_st', inplace = True)

#Convert index to local standard time
goes.index = goes.index.tz_convert(offset)

In [ ]:
#Convert time to pandas datetime
goes_sw.loc [:,'stimestamp'] = pd.to_datetime (goes_sw.loc [:, 'stimestamp'])
goes_sw.loc [:,'etimestamp'] = pd.to_datetime (goes_sw.loc [:, 'etimestamp'])

#Make a new column which will turn into the index and then be converted to tonzi time
goes_sw.loc[:, 'etimestamp_local_st'] = goes_sw.loc[:, 'etimestamp']

#Set index as local standard time
goes_sw.set_index('etimestamp_local_st', inplace = True)

#Convert goes dsr index to local standard time
goes_sw.index = goes_sw.index.tz_convert(offset)

In [ ]:
#Add UTC offset to tonzi stimestamp string
tonzi.loc[:,'TIMESTAMP_START_utcoffset'] = (tonzi.loc[:,'TIMESTAMP_START']).astype(str) + amf_offset

#Convert stimestamp to datetime
tonzi.loc[:,'TIMESTAMP_START_utcoffset'] = pd.to_datetime(tonzi.loc[:,'TIMESTAMP_START_utcoffset'], 
                                                          format = '%Y%m%d%H%M%z')

#Add UTC offset to tonzi etimestamp
tonzi.loc[:,'TIMESTAMP_END_utcoffset'] = (tonzi.loc[:,'TIMESTAMP_END']).astype(str) + amf_offset

#Convert etimestamp to datetime
tonzi.loc[:,'TIMESTAMP_END_utcoffset'] = pd.to_datetime(tonzi.loc[:,'TIMESTAMP_END_utcoffset'], 
                                                        format = '%Y%m%d%H%M%z')

#Get midpoint timestamp
tonzi.loc[:, 'itime'] = tonzi.loc[:, 'TIMESTAMP_START_utcoffset'] + (tonzi.loc[:, 'TIMESTAMP_END_utcoffset'] - tonzi.loc[:, 'TIMESTAMP_START_utcoffset']) / 2

#Make a new column which will turn into the index and then be converted to tonzi time
tonzi.loc[:, 'itime_local_st'] = tonzi.loc[:, 'itime']

#Set itime_local_st as index
tonzi.set_index ('itime_local_st', inplace = True)

#Convert index to local standard time
tonzi.index = tonzi.index.tz_convert(offset)

#Retain original tonzi as well
tonzi_orig = tonzi

In [ ]:
#Add UTC offset to tonzi spectral data's timestamp
tonzi_spec.loc[:,'DateTime_utcoffset'] = (tonzi_spec.loc[:,'DateTime']).astype(str) + amf_offset

#Convert time to pandas datetime
tonzi_spec.loc[:,'DateTime_utcoffset'] = pd.to_datetime(tonzi_spec.loc[:,'DateTime_utcoffset'], 
                                                        format = '%m/%d/%y %H:%M%z')

#Make a new column which will turn into the index and then be converted to tonzi time
tonzi_spec.loc[:,'DateTime_local_st'] = tonzi_spec.loc[:,'DateTime_utcoffset']

#Set index
tonzi_spec.set_index ('DateTime_local_st', inplace = True)

#Convert index to local standard time
tonzi_spec.index = tonzi_spec.index.tz_convert(offset)

In [ ]:
goes_sw.drop_duplicates(inplace = True)

In [ ]:
#Set working directory where all figures and tables will be saved.
os.chdir(r'path to directory')

# Table 1: Get number of clear sky and good quality observations by season and product

In [ ]:
#Add month to dsr
goes_sw['month'] = goes_sw.index.month

#Initiate dataframe to populate
obs_dat = pd.DataFrame()

In [ ]:
seasons = ['winter', 'spring', 'summer', 'fall']

for season in seasons:
    if season == 'winter':
        l1b_mask = (goes['month'] >=1) & (goes['month'] <=3)
        dsr_mask = (goes_sw['month'] >=1) & (goes_sw['month'] <=3) & (goes_sw.DQF == 0.0)
        
    if season == 'spring':
        l1b_mask = (goes['month'] >=4) & (goes['month'] <=6)
        dsr_mask = (goes_sw['month'] >=4) & (goes_sw['month'] <=6) & (goes_sw.DQF == 0.0)
        
    if season == 'summer':
        l1b_mask = (goes['month'] >=7) & (goes['month'] <=9)
        dsr_mask = (goes_sw['month'] >=7) & (goes_sw['month'] <=9) & (goes_sw.DQF == 0.0)
        
    if season == 'fall':
        l1b_mask = (goes['month'] >=10) & (goes['month'] <=12)
        dsr_mask = (goes_sw['month'] >=10) & (goes_sw['month'] <=12) & (goes_sw.DQF == 0.0)
        
    for sensor in [16, 17]:
        obs_dat.loc[season + str(sensor), 'season'] = season
        obs_dat.loc[season + str(sensor), 'sensor'] = sensor
        obs_dat.loc[season + str(sensor), 'l1b_n_clear_goodq'] = goes.loc[l1b_mask & (goes.sensor == sensor)].shape[0]
        obs_dat.loc[season + str(sensor), 'dsr_n_clear_goodq'] = goes_sw.loc[dsr_mask & (goes_sw.sensor == sensor)].shape[0]

                

In [ ]:
obs_dat

In [ ]:
#Export table
# obs_dat.to_csv(r'number_obs.csv',
#                index = False)

# Calculate doy, decimal hour, and solar zenith angle

In [ ]:
#Get hour and doy as decimal hour for sza calculation
goes.loc[:, 'dec_hour'] = goes.index.hour + (goes.index.minute / 60) + (goes.index.second / 3600)
tonzi.loc[:, 'dec_hour'] = tonzi.index.hour + (tonzi.index.minute / 60) + (tonzi.index.second / 3600)
goes.loc[:, 'doy'] = goes.index.dayofyear
tonzi.loc[:, 'doy'] = tonzi.index.dayofyear

In [ ]:
#Limit tonzi to goes dates before calculating sza
tonzi = tonzi['2019-01-01': '2020-12-31'].copy()

In [ ]:
#Drop duplicates in goes 2
goes.sort_values(by  = ['etimestamp_local_st', 'sensor'], inplace = True) #Sort by index and sensor so goes17 is later entry 
goes.drop_duplicates(subset = 'stimestamp_x', keep = 'last', inplace = True) #Keep goes17 if duplicate timestamps exist

In [ ]:
#Calculate sza for goes if there is no sza and saa included in goes
#Get solar angles: SZA, SAA
lat = goes['locationDecimalLatitude'][0]
lon = goes['locationDecimalLongitude'][0]
goes.loc[:, 'sza'] = pv.solarposition.spa_python(goes.index, lat, lon).zenith
goes.loc[:, 'saa'] = pv.solarposition.spa_python(goes.index, lat, lon).azimuth

In [ ]:
#Calculate sza for tonzi
#Get solar angles: SZA, SAA
tonzi.loc[:, 'sza'] = pv.solarposition.spa_python(tonzi.index, lat, lon).zenith
tonzi.loc[:, 'saa'] = pv.solarposition.spa_python(tonzi.index, lat, lon).azimuth

# Calculate vegetation indices 

In [ ]:
#Filter for sza
goes = goes.loc[(goes.sza < 70)].copy()
tonzi2 = tonzi.copy() #Keep copy of unfiltered one for plotting
tonzi = tonzi.loc[(tonzi.sza < 70)].copy()

In [ ]:
#Calculate NDVI: eq. 6
goes.loc[:, 'ndvi_rdd_0vza'] = ((goes.loc[:, 'band3_r_dd_iv_0vza'] - goes.loc[:, 'band2_r_dd_iv_0vza']) 
                            / (goes.loc[:, 'band3_r_dd_iv_0vza'] + goes.loc[:, 'band2_r_dd_iv_0vza'])
                           )
goes.loc[:, 'ndvi_toa'] = ((goes.loc[:, 'band3_ref'] - goes.loc[:, 'band2_ref']) 
                            / (goes.loc[:, 'band3_ref'] + goes.loc[:, 'band2_ref'])
                           )

#Calculate NIRv from reflectance: eq. 12
ndvi_rdd_0vza_mask = (goes.loc[:, 'ndvi_rdd_0vza'] >= 0) & (goes.loc[:, 'ndvi_rdd_0vza'] <= 1)
goes.loc[ndvi_rdd_0vza_mask, 'nirv_rdd_0vza'] = (goes.loc[ndvi_rdd_0vza_mask, 'band3_r_dd_iv_0vza']
                                             * goes.loc[ndvi_rdd_0vza_mask, 'ndvi_rdd_0vza'])

ndvi_toa_mask = (goes.loc[:, 'ndvi_toa'] >= 0) & (goes.loc[:, 'ndvi_toa'] <= 1)
goes.loc[ndvi_toa_mask, 'nirv_toa'] = (goes.loc[ndvi_toa_mask, 'band3_ref'] 
                                        * goes.loc[ndvi_toa_mask, 'ndvi_toa'])

In [ ]:
#Calculate reflectance from tonzi tower spectral data
tonzi_spec.loc[tonzi_spec.NDVI_650in > 0, 'Red_ref_tower'] = (tonzi_spec.loc[tonzi_spec.NDVI_650in > 0, 'NDVI_650out_all'] 
                                                              / tonzi_spec.loc[tonzi_spec.NDVI_650in > 0, 'NDVI_650in']
                                                             )
tonzi_spec.loc[tonzi_spec.NDVI_810in > 0, 'NIR_ref_tower'] = (tonzi_spec.loc[tonzi_spec.NDVI_810in > 0, 'NDVI_810out_all'] 
                                                              / tonzi_spec.loc[tonzi_spec.NDVI_810in > 0, 'NDVI_810in']
                                                             )

#Calculate NDVI
tonzi_spec.loc[tonzi_spec.NIR_ref_tower > 0, 'NDVI_tower'] = ((tonzi_spec.loc[tonzi_spec.NIR_ref_tower > 0, 'NIR_ref_tower'] 
                                                               - tonzi_spec.loc[tonzi_spec.NIR_ref_tower > 0, 'Red_ref_tower']) 
                                                              / (tonzi_spec.loc[tonzi_spec.NIR_ref_tower > 0, 'NIR_ref_tower'] 
                                                                 + tonzi_spec.loc[tonzi_spec.NIR_ref_tower > 0, 'Red_ref_tower'])
                                                             )
#NIRv based on NIR outgoing radiance
tonzi_spec.loc[tonzi_spec.NDVI_tower >= 0, 'NIRv_rad_tower'] = (tonzi_spec.loc[tonzi_spec.NDVI_tower >= 0, 'NDVI_810out_all']
                                                                * tonzi_spec.loc[tonzi_spec.NDVI_tower >= 0, 'NDVI_tower'])

#NIRv based on NIR reflectance
tonzi_spec.loc[tonzi_spec.NDVI_tower >= 0, 'NIRv_ref_tower'] = (tonzi_spec.loc[tonzi_spec.NDVI_tower >= 0, 'NIR_ref_tower'] 
                                                                * tonzi_spec.loc[tonzi_spec.NDVI_tower >= 0, 'NDVI_tower'])

In [ ]:
#Calculate midday median vegetation indices
goes.loc[:, 'date'] = goes.index.date
goes_med = goes.loc[(goes.dec_hour > 10) & (goes.dec_hour < 14)].groupby('date').median()

# Interpolate SW  between the hours to match tonzi half hourly resolution

In [ ]:
#Take floor of timestamp so resampling intervals are consistent
dsr = goes_sw.loc[(goes_sw.DQF == 0.0) & (goes_sw.sensor == 17)].sort_index().copy() #Pick sensor and get good quality shortwave
dsr.index = dsr.index.floor('H')

#Upsample dsr to half hour
dsr_30 = pd.DataFrame(dsr['DSR']).sort_index().resample('30T', 
                                                        closed = 'left', 
                                                        label = 'left').interpolate('linear', 
                                                                                    limit_area = 'inside')

#Add PAR (W/m2) to goes tonzi: eq. 7
dsr_30.loc[:, 'PAR'] = 0.45 * dsr_30.loc[:, 'DSR'] #W/m2 = J/s/m2

#Convert PAR in W/m2 (J/s/m2) to PPFD in µmolPhoton m-2 s-1 using Table 3 conversion rates from Thimijan and Heins, 1983
#eq. 20
dsr_30.loc[:, 'PPFD'] = 4.57 * dsr_30.loc[:, 'PAR']

#Add hour, date and doy column
dsr_30.loc[:, 'date'] = dsr_30.index.date
dsr_30.loc[:, 'doy'] = dsr_30.index.dayofyear
dsr_30.loc[:, 'hour'] = dsr_30.index.hour
dsr_30.loc[:, 'time'] = dsr_30.index.time

In [ ]:
#Add midday median VIs from GOES to dsr_30 to calculate NIRvP or NDVIP later
for date in np.unique(dsr_30.date):
    if date in goes_med.index:
        
        ndvi_rdd_0vza = goes_med.loc[date, 'ndvi_rdd_0vza']
        dsr_30.loc[dsr_30.date == date, 'med_ndvi_rdd_0vza'] = ndvi_rdd_0vza
        
        ndvi_toa = goes_med.loc[date, 'ndvi_toa']
        dsr_30.loc[dsr_30.date == date, 'med_ndvi_toa'] = ndvi_toa


for date in np.unique(dsr_30.date):
    if date in goes_med.index:
        
        nirv_rdd_0vza = goes_med.loc[date, 'nirv_rdd_0vza']
        dsr_30.loc[dsr_30.date == date, 'med_nirv_rdd_0vza'] = nirv_rdd_0vza
        
        nirv_toa = goes_med.loc[date, 'nirv_toa']
        dsr_30.loc[dsr_30.date == date, 'med_nirv_toa'] = nirv_toa

In [ ]:
#Check values
dsr_30

# Merge all data

In [ ]:
#Match closest date times between tonzi spectral and tonzi tower data
tonzi3 = pd.merge_asof(tonzi.sort_index(), 
                       tonzi_spec[['NDVI_650in',
                                   'NDVI_810in',
                                   'NDVI_650out_all',
                                   'NDVI_810out_all',
                                   'Red_ref_tower',
                                   'NIR_ref_tower',
                                   'NDVI_tower',
                                   'NIRv_rad_tower',
                                   'NIRv_ref_tower',
                                   'hour',
                                   'DateTime']].sort_index(), 
                       left_index = True, 
                       right_index = True, 
                       allow_exact_matches = True,
                       direction = 'backward', #joins 8:00 to 8:15, 9:30 to 9:45...as opposed to forward (9:00 to 8:45)
                       suffixes = ('', '_tower'),
                       tolerance = pd.Timedelta("0 days 00:16:00")
                          )

In [ ]:
#Check date joins
tonzi3[['dec_hour','NIRv_ref_tower', 'hour', 'DateTime']]

In [ ]:
#Match closest dates between dsr_30 and tonzi3..
goes_tonzi = pd.merge_asof(tonzi3.sort_index(),
                            dsr_30[['DSR', 
                                    'PAR',
                                    'PPFD',
                                    'med_ndvi_rdd_0vza', 
                                    'med_ndvi_toa', 
                                    'med_nirv_rdd_0vza', 
                                    'med_nirv_toa',
                                    'hour',
                                    'doy',
                                    'date',
                                    'time']].sort_index(),
                              left_index = True,
                              right_index = True,
                              allow_exact_matches = True,
                              direction = 'forward',
                              suffixes = ('', '_sw'),
                              tolerance = pd.Timedelta("0 days 00:15:00")
                          )

In [ ]:
#Check the date join
goes_tonzi[['dec_hour', 'SW_IN_1_1_1','DSR', 'med_nirv_rdd_0vza', 'hour', 'date', 'time', 'DateTime']]

In [ ]:
len(tonzi_orig), len(tonzi3), len(goes_tonzi)

In [ ]:
#Remove data tables not needed anymore
del dsr_30
del goes_sw 
del goes_med

In [ ]:
#Set plotting style to match diurnal centroids plot
sb.set_style ("white")

# Set all fields according to the height of instrumentation being used

In [ ]:
t_air_field = 'TA_PI_F_1_1_1'
vpd_field = 'VPD_PI_1_1_1'
le_field = 'LE_1_1_1'
h_field = 'H_1_1_1'

#CO2 flux fields
gpp_field = 'GPP_PI_F_1_1_1'
nee_field = 'NEE_PI_F_1_1_1'
reco_field = 'RECO_PI_F_1_1_1'

#Set incoming shortwave field
sw_field = 'SW_IN_1_1_1'

In [ ]:
#Sort indices
goes_tonzi.sort_index(inplace = True)
tonzi.sort_index(inplace = True)

In [ ]:
#Get midday median tower NIRv_ref
tonzi_med = goes_tonzi.loc[(goes_tonzi.dec_hour > 10) & (goes_tonzi.dec_hour < 14)].groupby('date').median()

for date in np.unique(goes_tonzi.loc[~goes_tonzi.date.isna(), 'date']):
    if date in tonzi_med.index:
        nirv = tonzi_med.loc[date, 'NIRv_ref_tower']
        goes_tonzi.loc[goes_tonzi.date == date, 'med_nirv_ref_tower'] = nirv
        ndvi = tonzi_med.loc[date, 'NDVI_tower']
        goes_tonzi.loc[goes_tonzi.date == date, 'med_ndvi_tower'] = ndvi

# Calcualte NIRvP and NDVIP and other variables needed

In [ ]:
#NDVIP: eq. 5 and 6
goes_tonzi.loc[:, 'apar_ndvi_rdd_0vza'] = (goes_tonzi.loc[:, 'med_ndvi_rdd_0vza'] * goes_tonzi.loc[:, 'PAR'])
goes_tonzi.loc[:, 'apar_ndvi_toa'] = (goes_tonzi.loc[:, 'med_ndvi_toa'] * goes_tonzi.loc[:, 'PAR'])

#NIRvP: eq. 11
goes_tonzi.loc[:, 'apar_nirv_rdd_0vza'] = (goes_tonzi.loc[:, 'med_nirv_rdd_0vza'] * goes_tonzi.loc[:, 'PAR'])
goes_tonzi.loc[:, 'apar_nirv_toa'] = (goes_tonzi.loc[:, 'med_nirv_toa'] * goes_tonzi.loc[:, 'PAR'])

#Calculate tower nirvp with midday median tower NIRv and goes-r nirvp with goes-r ppfd
goes_tonzi.loc[:, 'apar_nirv_ref_tower'] = goes_tonzi.loc[:, 'med_nirv_ref_tower'] * goes_tonzi.loc[:, 'PPFD_IN_1_1_1']
goes_tonzi.loc[:, 'apar_nirv_rdd_0vza_ppfd'] = (goes_tonzi.loc[:, 'med_nirv_rdd_0vza'] * goes_tonzi.loc[:, 'PPFD'])
goes_tonzi.loc[:, 'apar_nirv_toa_ppfd'] = (goes_tonzi.loc[:, 'med_nirv_toa'] * goes_tonzi.loc[:, 'PPFD'])

#Calculate tower ndvip with midday median tower NDVI and goes-r ndvip with goes-r ppfd
goes_tonzi.loc[:, 'apar_ndvi_ref_tower'] = goes_tonzi.loc[:, 'med_ndvi_tower'] * goes_tonzi.loc[:, 'PPFD_IN_1_1_1']
goes_tonzi.loc[:, 'apar_ndvi_rdd_0vza_ppfd'] = (goes_tonzi.loc[:, 'med_ndvi_rdd_0vza'] * goes_tonzi.loc[:, 'PPFD'])
goes_tonzi.loc[:, 'apar_ndvi_toa_ppfd'] = (goes_tonzi.loc[:, 'med_ndvi_toa'] * goes_tonzi.loc[:, 'PPFD'])

In [ ]:
#Add month
goes_tonzi.loc[:, 'month'] = goes_tonzi.index.month

# Estimate parameters for LUE-NDVI, LIN-NIRvP, and LRC-NIRvP

In [ ]:
#Set the vegetation index field to be used
modis_apar = 'apar_ndvi_rdd_0vza'
nirv_apar = 'apar_nirv_rdd_0vza'

In [ ]:
def find_luendvip (params, x_dat, return_tscale = False, return_vpdscale = False):
    
    x_dat.sort_index(inplace = True)
    #Calculate modis temperature and vpd scalar
#     t_min = params[0]
#     t_max = params[1]
#     vpd_min = params[2]
#     vpd_max = params[3]
    
    #When VPD stress is active
    vpdscal_mask = (x_dat[vpd_field] > params[2]) & (x_dat[vpd_field] < params[3])
    #When VPD stress in not active
    vpd_opt_mask = x_dat[vpd_field] <= params[2]
    
    #When temperature stress is active
    tscal_mask = (x_dat[t_air_field] > params[0]) & (x_dat[t_air_field] < params[1])
    #When temperature stress is not active
    t_opt_mask = (x_dat[t_air_field] >= params[1])
    
    #eq. 9
    x_dat.loc[vpd_opt_mask, 'mod_vpdscale'] = 1
    x_dat.loc[x_dat[vpd_field] >= params[3], 'mod_vpdscale'] = 0
    x_dat.loc[vpdscal_mask, 'mod_vpdscale'] = ((params[3] - x_dat.loc[vpdscal_mask, vpd_field])
                                                / (params[3] - params[2])
                                              )
    #eq. 8
    x_dat.loc[t_opt_mask, 'mod_tscale'] = 1
    x_dat.loc[x_dat[t_air_field] <= params[0], 'mod_tscale'] = 0
    
    x_dat.loc[tscal_mask, 'mod_tscale'] = ((x_dat.loc[tscal_mask, t_air_field] - params[0])
                                           / (params[1] - params[0])
                                          )
    
    if return_tscale:
        return (x_dat.loc[:, 'mod_tscale'])
    
    if return_vpdscale:
        return (x_dat.loc[:, 'mod_vpdscale'])
    
    #eq. 4
    return (params[4] * x_dat.loc[:,'mod_tscale'] * x_dat.loc[:,'mod_vpdscale'] * x_dat[modis_apar])

def resid_luendvip (params, x_dat, y_dat):
    return (find_luendvip(params, x_dat) - y_dat)

In [ ]:
def find_lin_nirvp (params, x_dat):
    x_dat.sort_index(inplace = True)
    #eq. 10
    return (params * x_dat[nirv_apar])

def resid_lin_nirvp (params, x_dat, y_dat):
    return (find_lin_nirvp(params, x_dat) - y_dat)

In [ ]:
def find_lrc_nirvp (params, x_dat, return_beta = False):
    x_dat.sort_index(inplace = True)
    grt_mask = x_dat[vpd_field] > 10
    less_mask = x_dat[vpd_field] <= 10
    #eq. 14
    x_dat.loc[grt_mask, 'beta'] = params[1] * (np.exp((-params[2] * (x_dat.loc[grt_mask, vpd_field] - 10)).astype('float128')))
    x_dat.loc[less_mask, 'beta'] = params[1]
    
    if return_beta:
        return (x_dat['beta'])
    
    #eq. 13
    return ((params[0] * x_dat.loc[:, nirv_apar] * x_dat.loc[:, 'beta']) 
            / (x_dat.loc[:, 'beta'] + (params[0] * x_dat.loc[:, nirv_apar])))

def resid_lrc_nirvp (params, x_dat, y_dat):
    return (find_lrc_nirvp(params, x_dat) - y_dat)

In [ ]:
months = np.unique(goes_tonzi.month).astype(str)

models = ['lue_ndvip', 'lin_nirvp', 'lrc_nirvp']

colnames = ['rmse_train', 'nrmse_train', 'me_train', 'mae_train', 'nmae_train',
            'rmse_test', 'nrmse_test', 'me_test', 'mae_test', 'nmae_test']

#Make empty dataframe which will be populated below
err_sums = pd.DataFrame()

#Make empty list which will be populated by tables generated below
month_tables = []

#test
#month = '1'

for month in months:
    print(month)

    #-----Split data into test and train sets---------------------------
    #Make mask for valid observations
    mask = (goes_tonzi[gpp_field] > 0) & (goes_tonzi[gpp_field] < 25) & (goes_tonzi[vpd_field] > 0) & (goes_tonzi.P < 10)

    #Make date mask according to month
    date_mask = goes_tonzi.month == int(month)

    #Make data for the month
    goes_tonzi_gs = goes_tonzi.loc[date_mask].loc[mask].sort_index()
    goes_tonzi_gs.sort_index(inplace = True)
        
    #Split data into training and test sets
    oth_mask = (goes_tonzi_gs[nirv_apar] >= 0)

    x_train_oth, x_test_oth, y_train_oth, y_test_oth = train_test_split(goes_tonzi_gs.loc[oth_mask].sort_index(), 
                                                                        goes_tonzi_gs.loc[oth_mask].sort_index(), 
                                                                        test_size = 0.30,
                                                                        random_state = 10
                                                                       )

    #------------Least squares for parameters-------------------------------
    #----Set initial values for parameters
    #LUE intitals from MODIS User Guide table for this biome
    eps_max_o = 0.03
    t_min = -8.0 #C = params[0]
    t_max = 11.39 #C = params[1]
    vpd_min = 650.0 * 0.01 #hPa = params[2]
    vpd_max = 3200.0 * 0.01 #hPa = params[3]
    
    #Lin-NIRvP
    eps_nirvp_o = (x_train_oth[gpp_field]
                   / x_train_oth[nirv_apar]).median()
    
    #LRC-NIRvP
    gpp_high_quant_mask = x_train_oth[gpp_field] >= x_train_oth[gpp_field].quantile(q=0.75) #highest values of gpp in data
    gpp_low_quant_mask = x_train_oth[gpp_field] <= x_train_oth[gpp_field].quantile(q=0.25) #lowest values of gpp in data (possibly during light limitations)
    vpd_mask = x_train_oth[vpd_field] > 10
    
    alpha_o = (x_train_oth.loc[gpp_low_quant_mask, gpp_field]
               / x_train_oth.loc[gpp_low_quant_mask, nirv_apar]).mean()
    beta_o = x_train_oth[gpp_field].max() 
    k = 0.03

    print ('alpha_o = ' + str(alpha_o), 'beta_o = ' + str(beta_o))


    #----Call least_sqaures with residuals
    #Set bounds: ([lower], [upper])
    lue_bounds = ([-np.inf, -np.inf, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])
    model_luendvip = optimize.least_squares(resid_luendvip, 
                                       x0 = [t_min, t_max, vpd_min, vpd_max, eps_max_o], 
                                       args = (x_train_oth, 
                                               y_train_oth[gpp_field]),
                                       bounds = lue_bounds,
                                       loss = 'huber'
                                      )
    
    lin_nirvp_bounds = ([0], [np.inf])
    model_lin_nirvp = optimize.least_squares(resid_lin_nirvp,
                                             x0= eps_nirvp_o,
                                             args = (x_train_oth, 
                                                     y_train_oth[gpp_field]),
                                             bounds = lin_nirvp_bounds,
                                             loss = 'huber'
                                            )
    
    lrc_nirvp_bounds = ([0, 0, -np.inf], [np.inf, np.inf, np.inf])
    model_lrcnirvp = optimize.least_squares(resid_lrc_nirvp, 
                                      x0 = [alpha_o, beta_o, k],
                                      args = (x_train_oth,
                                              y_train_oth[gpp_field]),
                                       bounds = lrc_nirvp_bounds,
                                       loss = 'huber'
                                  )
    print ('lrc-nirvp done')

    #-------------Get error summaries---------------------------------
    for model in models:
        
        #Get training and testing estimated GPP and add model parameters to error summaries table
        if model == 'lue_ndvip':
            train_pred_gpp = find_luendvip (params = model_luendvip.x,
                                                x_dat = x_train_oth)
            test_pred_gpp = find_luendvip (params = model_luendvip.x,
                                                x_dat = x_test_oth)
            
            #Add model and month to error summaries table
            err_sums.loc[model + month, 'month'] = month
            err_sums.loc[model + month, 'model'] = model
            
            #Add initial values and model parameters to error summaries table
            err_sums.loc[model + month, 'lue_tmin'] = model_luendvip.x[0]
            err_sums.loc[model + month, 'lue_tmax'] = model_luendvip.x[1]
            err_sums.loc[model + month, 'lue_vpdmin'] = model_luendvip.x[2]
            err_sums.loc[model + month, 'lue_vpdmax'] = model_luendvip.x[3]
            err_sums.loc[model + month, 'eps_max'] = model_luendvip.x[4]
            
            err_sums.loc[model + month, 'lue_tmin_o'] = t_min
            err_sums.loc[model + month, 'lue_tmax_o'] = t_max
            err_sums.loc[model + month, 'lue_vpdmin_o'] = vpd_min
            err_sums.loc[model + month, 'lue_vpdmax_o'] = vpd_max
            err_sums.loc[model + month, 'eps_max_o'] = eps_max_o
            
        if model == 'lin_nirvp':
            train_pred_gpp = find_lin_nirvp (params = model_lin_nirvp.x,
                                                x_dat = x_train_oth)
            test_pred_gpp = find_lin_nirvp (params = model_lin_nirvp.x,
                                                x_dat = x_test_oth)
            
            #Add model and month to error summaries table
            err_sums.loc[model + month, 'month'] = month
            err_sums.loc[model + month, 'model'] = model

            #Add model parameters and initial values to error summaries table
            err_sums.loc[model + month, 'eps_nirvp'] = model_lin_nirvp.x
            err_sums.loc[model + month, 'eps_nirvp_o'] = eps_nirvp_o
            
        if model == 'lrc_nirvp':
            train_pred_gpp = find_lrc_nirvp (params = model_lrcnirvp.x,
                                             x_dat = x_train_oth)
            test_pred_gpp = find_lrc_nirvp (params = model_lrcnirvp.x,
                                            x_dat = x_test_oth)
            
            #Add model and month to error summaries table
            err_sums.loc[model + month, 'month'] = month
            err_sums.loc[model + month, 'model'] = model

            #Add model parameters and initial values to error summaries table
            err_sums.loc[model + month, 'alpha'] = model_lrcnirvp.x[0]
            err_sums.loc[model + month, 'beta_o'] = model_lrcnirvp.x[1]
            err_sums.loc[model + month, 'k'] = model_lrcnirvp.x[2]
            
            err_sums.loc[model + month, 'alpha_o'] = alpha_o
            err_sums.loc[model + month, 'beta_o_o'] = beta_o
            err_sums.loc[model + month, 'k_o'] = k
           
        #Set train and test tower gpp
        train_tower_gpp = y_train_oth[gpp_field]
        test_tower_gpp = y_test_oth[gpp_field]
        
        
        #----Calculate RMSE, NRMSE, ME, MAE, NMAE
        errors_train = train_pred_gpp - train_tower_gpp
        me_train = errors_train.mean()
        rmse_train = np.sqrt((errors_train ** 2).mean())
        nrmse_train = rmse_train / train_tower_gpp.mean()
        mae_train = abs(errors_train).mean()
        nmae_train = mae_train / train_tower_gpp.mean()

        errors_test = test_pred_gpp - test_tower_gpp
        me_test = errors_test.mean()
        rmse_test = np.sqrt((errors_test ** 2).mean())
        nrmse_test = rmse_test / test_tower_gpp.mean()
        mae_test = abs(errors_test).mean()
        nmae_test = mae_test / test_tower_gpp.mean()

        mets = {'rmse_train':rmse_train,
                 'nrmse_train': nrmse_train,
                 'me_train':me_train,
                 'mae_train': mae_train,
                 'nmae_train': nmae_train,
                 'rmse_test':rmse_test,
                 'nrmse_test': nrmse_test,
                 'me_test': me_test,
                 'mae_test': mae_test,
                 'nmae_test': nmae_test
                }

        #Add error summaries to table
        for col in colnames:
            err_sums.loc[model + month, col] = mets[col]

    print ('Parameters estimated')
    
    
    #------------Make month's table of estimates----------------------------
    oth_mask_gs = (goes_tonzi_gs[nirv_apar] >= 0)


    #---LUE NDVI
    #Estimate values based on fitted parameters
    goes_tonzi_gs.loc[oth_mask_gs, 'lue_ndvip'] = find_luendvip (params = model_luendvip.x,
                                                                x_dat = goes_tonzi_gs.loc[oth_mask_gs].copy())
    #Add Tscale and Wscale
    goes_tonzi_gs.loc[oth_mask_gs, 'lue_tscale'] = find_luendvip (params = model_luendvip.x,
                                                                  x_dat = goes_tonzi_gs.loc[oth_mask_gs].copy(),
                                                                  return_tscale = True
                                                                 )
    goes_tonzi_gs.loc[oth_mask_gs, 'lue_vpdscale'] = find_luendvip (params = model_luendvip.x,
                                                                  x_dat = goes_tonzi_gs.loc[oth_mask_gs].copy(),
                                                                  return_vpdscale = True
                                                                 )
    goes_tonzi_gs.loc[oth_mask_gs, 'lue_tmin'] = model_luendvip.x[0]
    goes_tonzi_gs.loc[oth_mask_gs, 'lue_tmax'] = model_luendvip.x[1]
    goes_tonzi_gs.loc[oth_mask_gs, 'lue_vpdmin'] = model_luendvip.x[2]
    goes_tonzi_gs.loc[oth_mask_gs, 'lue_vpdmax'] = model_luendvip.x[3]
    goes_tonzi_gs.loc[oth_mask_gs, 'lue_eps_max'] = model_luendvip.x[4]

    #---Linear NIRvP
    #Estimate values based on fitted parameters
    goes_tonzi_gs.loc[oth_mask_gs, 'lin_nirvp'] = find_lin_nirvp (params = model_lin_nirvp.x,
                                                                      x_dat = goes_tonzi_gs.loc[oth_mask_gs].copy())


    #---LRC NIRvP
    #Estimate values based on fitted parameters
    goes_tonzi_gs.loc[oth_mask_gs, 'beta'] = find_lrc_nirvp (params = model_lrcnirvp.x,
                                                             x_dat = goes_tonzi_gs.loc[oth_mask_gs].copy(),
                                                             return_beta = True)
    goes_tonzi_gs.loc[oth_mask_gs, 'lrc_nirvp'] = find_lrc_nirvp (params = model_lrcnirvp.x,
                                                                     x_dat = goes_tonzi_gs.loc[oth_mask_gs].copy())

    #------Mark train or test data and record sample size------------------------
    goes_tonzi_gs.loc[goes_tonzi_gs.index.isin(x_train_oth.index), 'train_oth'] = '1'
    goes_tonzi_gs.loc[goes_tonzi_gs.index.isin(x_test_oth.index), 'test_oth'] = '1'
    
    goes_tonzi_gs.loc[goes_tonzi_gs.index.isin(x_train_oth.index), 'train_oth_n'] =  len(goes_tonzi_gs.loc[goes_tonzi_gs.train_oth == '1'])
    goes_tonzi_gs.loc[goes_tonzi_gs.index.isin(x_test_oth.index), 'test_oth_n'] = len(goes_tonzi_gs.loc[goes_tonzi_gs.test_oth == '1'])
    
    #-----Add to month_tables----------------------------------------------
    month_tables.append(goes_tonzi_gs)

In [ ]:
#err_sums

In [ ]:
len(month_tables)

In [ ]:
#Export error table 
# err_sums.to_csv('monthly_err_sums.csv', 
#                 header = True, 
#                 index = False)

In [ ]:
#combine tables and sort index
goes_tonzi_gs = pd.concat(month_tables)
goes_tonzi_gs.sort_index(inplace = True)

# Table 2: seasonal error summaries

In [ ]:
#Calculate error metrics for seasons

seasons = ['winter', 'spring', 'summer', 'fall']
models = ['lue_ndvip', 'lin_nirvp', 'lrc_nirvp']

colnames = ['rmse_train', 'nrmse_train', 'me_train', 'mae_train', 'nmae_train',
            'rmse_test', 'nrmse_test', 'me_test', 'mae_test', 'nmae_test']
seas_err_sums = pd.DataFrame()

#Make empty list which will be populated by tables generated below
month_tables = []

#test
#season = 'spring'

for season in seasons:
    #-----Split data into test and train sets

    #Make season
    if season == 'winter':
        date_mask = ((goes_tonzi_gs.month <= 3) & (goes_tonzi_gs.month >= 1))
    if season == 'spring':
        date_mask = ((goes_tonzi_gs.month <= 6) & (goes_tonzi_gs.month >= 4))
    if season == 'summer':
        date_mask = ((goes_tonzi_gs.month <= 9) & (goes_tonzi_gs.month >= 7))
    if season == 'fall':
        date_mask = ((goes_tonzi_gs.month <= 12) & (goes_tonzi_gs.month >= 10))
    
    for model in models:
        #Set train and test tower gpp
        train_mask = goes_tonzi_gs.train_oth == '1'
        test_mask = goes_tonzi_gs.test_oth == '1'
        
        train_pred_gpp = goes_tonzi_gs.loc[date_mask].loc[train_mask, model]
        test_pred_gpp = goes_tonzi_gs.loc[date_mask].loc[test_mask, model]
        
        train_tower_gpp = goes_tonzi_gs.loc[date_mask].loc[train_mask, gpp_field]
        test_tower_gpp = goes_tonzi_gs.loc[date_mask].loc[test_mask, gpp_field]
        
        #----Calculate RMSE, NRMSE, ME, MAE, NMAE
        errors_train = train_pred_gpp - train_tower_gpp
        me_train = errors_train.mean()
        rmse_train = np.sqrt((errors_train ** 2).mean())
        nrmse_train = rmse_train / train_tower_gpp.mean()
        mae_train = abs(errors_train).mean()
        nmae_train = mae_train / train_tower_gpp.mean()

        errors_test = test_pred_gpp - test_tower_gpp
        me_test = errors_test.mean()
        rmse_test = np.sqrt((errors_test ** 2).mean())
        nrmse_test = rmse_test / test_tower_gpp.mean()
        mae_test = abs(errors_test).mean()
        nmae_test = mae_test / test_tower_gpp.mean()

        mets = {'rmse_train':rmse_train,
                 'nrmse_train': nrmse_train,
                 'me_train':me_train,
                 'mae_train': mae_train,
                 'nmae_train': nmae_train,
                 'rmse_test':rmse_test,
                 'nrmse_test': nrmse_test,
                 'me_test': me_test,
                 'mae_test': mae_test,
                 'nmae_test': nmae_test
                }

        #Add error summaries to table
        seas_err_sums.loc[model + season, 'season'] = season
        seas_err_sums.loc[model + season, 'model'] = model
        for col in colnames:
            seas_err_sums.loc[model + season, col] = mets[col]


In [ ]:
#seas_err_sums

In [ ]:
#Export error table and concatenate seasonal tables
# seas_err_sums.to_csv('seas_err_sums.csv', 
#                 header = True, 
#                 index = False)

# Figure 1

In [ ]:
#Set up plot for light response
fig, axs = plt.subplots(4, 3, figsize=(15,17), constrained_layout = True)

#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)

#------Winter------------------------------------
dat_mask = (goes_tonzi_gs.month >= 1) & (goes_tonzi_gs.month <= 3)
dat_mask = (goes_tonzi_gs.month == 2) 
plt_mask = (goes_tonzi_gs[nirv_apar] >= 0) & (goes_tonzi_gs.PPFD_IN_1_1_1 >= 0) #filler mask
season = '2'

#Set data
gpp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
ppfd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'PPFD_IN_1_1_1'].sort_index()
nirvp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , nirv_apar].sort_index()
ndvip_scal = (goes_tonzi_gs.loc[dat_mask].loc[plt_mask , modis_apar].sort_index()
              * goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'lue_tscale'].sort_index()
              * goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'lue_vpdscale'].sort_index()
             )
nirvp_pred = np.array([f for f in range(0, 200)])
ndvip_scal_pred = np.array([f for f in range(0, 200)])

lrc_nirvp_fit = ((err_sums.loc['lrc_nirvp' + season, 'alpha'] 
                 * nirvp_pred 
                  * err_sums.loc['lrc_nirvp' + season, 'beta_o'])
                 / (err_sums.loc['lrc_nirvp' + season, 'beta_o']
                    + (err_sums.loc['lrc_nirvp' + season, 'alpha'] 
                       * nirvp_pred)
              )
                )

lin_nirvp_fit = (err_sums.loc['lin_nirvp' + season, 'eps_nirvp'] * nirvp_pred)

lue_fit = (err_sums.loc['lue_ndvip' + season, 'eps_max'] 
           * ndvip_scal_pred
          ) 

#Set up plot
axs[0, 0].set_xlim([-1, max(ndvip_scal) + 3])
axs[0, 0].set_ylim([-1, max(gpp) + 3])

axs[0, 1].set_xlim([-1, max(nirvp) + 3])
axs[0, 1].set_ylim([-1, max(gpp) + 3])

axs[0, 2].set_xlim([-1, max(ppfd) + 3])
axs[0, 2].set_ylim([-1, max(gpp) + 3])

#KDE
z_gpp_ndvip = gaussian_kde(np.vstack([gpp, ndvip_scal]))(np.vstack([gpp, ndvip_scal]))
z_gpp_nirvp = gaussian_kde(np.vstack([gpp, nirvp]))(np.vstack([gpp, nirvp]))
z_gpp_ppfd = gaussian_kde(np.vstack([gpp, ppfd]))(np.vstack([gpp, ppfd]))

im1 = axs[0, 0].scatter (ndvip_scal,
                      gpp,
                      c = z_gpp_ndvip,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[0, 0].plot(ndvip_scal_pred,
                lue_fit,
                color = 'black',
               linewidth = 4
               )


im2 = axs[0, 1].scatter (nirvp,
                      gpp,
                      c = z_gpp_nirvp,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[0, 1].plot(nirvp_pred,
                lin_nirvp_fit,
                color = 'black',
                   linewidth = 4,
               linestyle = 'dashed'
               )
axs[0, 1].plot(nirvp_pred,
                lrc_nirvp_fit,
                color = 'black',
                linewidth = 4
               )

im3 = axs[0, 2].scatter (ppfd,
                      gpp,
                      c = z_gpp_ppfd,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

#------Spring------------------------------------

dat_mask = (goes_tonzi_gs.month >= 4) & (goes_tonzi_gs.month <= 6)
dat_mask = (goes_tonzi_gs.month == 5)
plt_mask = (goes_tonzi_gs[nirv_apar] >= 0) & (goes_tonzi_gs.PPFD_IN_1_1_1 >= 0) #filler mask

season = '5'

#Set data
gpp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
ppfd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'PPFD_IN_1_1_1'].sort_index()
nirvp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , nirv_apar].sort_index()
ndvip_scal = (goes_tonzi_gs.loc[dat_mask].loc[plt_mask , modis_apar].sort_index()
              * goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'lue_tscale'].sort_index()
              * goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'lue_vpdscale'].sort_index()
             )
nirvp_pred = np.array([f for f in range(0, 200)])
ndvip_scal_pred = np.array([f for f in range(0, 200)])

lrc_nirvp_fit = ((err_sums.loc['lrc_nirvp' + season, 'alpha'] 
                 * nirvp_pred 
                  * err_sums.loc['lrc_nirvp' + season, 'beta_o'])
                 / (err_sums.loc['lrc_nirvp' + season, 'beta_o']
                    + (err_sums.loc['lrc_nirvp' + season, 'alpha'] 
                       * nirvp_pred)
              )
                )

lin_nirvp_fit = (err_sums.loc['lin_nirvp' + season, 'eps_nirvp'] * nirvp_pred)

lue_fit = (err_sums.loc['lue_ndvip' + season, 'eps_max'] 
           * ndvip_scal_pred
          ) 

#Set up plot
axs[1, 0].set_xlim([-1, max(ndvip_scal) + 3])
axs[1, 0].set_ylim([-1, max(gpp) + 3])

axs[1, 1].set_xlim([-1, max(nirvp) + 3])
axs[1, 1].set_ylim([-1, max(gpp) + 3])

axs[1, 2].set_xlim([-1, max(ppfd) + 3])
axs[1, 2].set_ylim([-1, max(gpp) + 3])

#KDE
z_gpp_ndvip = gaussian_kde(np.vstack([gpp, ndvip_scal]))(np.vstack([gpp, ndvip_scal]))
z_gpp_nirvp = gaussian_kde(np.vstack([gpp, nirvp]))(np.vstack([gpp, nirvp]))
z_gpp_ppfd = gaussian_kde(np.vstack([gpp, ppfd]))(np.vstack([gpp, ppfd]))

im4 = axs[1, 0].scatter (ndvip_scal,
                      gpp,
                      c = z_gpp_ndvip,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.7
                     )

axs[1, 0].plot(ndvip_scal_pred,
                lue_fit,
                color = 'black',
                linewidth = 4
               )


im5 = axs[1, 1].scatter (nirvp,
                      gpp,
                      c = z_gpp_nirvp,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[1, 1].plot(nirvp_pred,
                lin_nirvp_fit,
                color = 'black',
                linewidth = 4,
               linestyle = 'dashed'
               )

axs[1, 1].plot(nirvp_pred,
                lrc_nirvp_fit,
                color = 'black',
                linewidth = 4
               )


im6 = axs[1, 2].scatter (ppfd,
                      gpp,
                      c = z_gpp_ppfd,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

#------Summer------------------------------------

dat_mask = (goes_tonzi_gs.month >= 7) & (goes_tonzi_gs.month <= 9)
dat_mask = (goes_tonzi_gs.month == 7)
plt_mask = (goes_tonzi_gs[nirv_apar] >= 0) & (goes_tonzi_gs.PPFD_IN_1_1_1 >= 0) #filler mask

season = '7'

#Set data
gpp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
ppfd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'PPFD_IN_1_1_1'].sort_index()
nirvp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , nirv_apar].sort_index()
ndvip_scal = (goes_tonzi_gs.loc[dat_mask].loc[plt_mask , modis_apar].sort_index()
              * goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'lue_tscale'].sort_index()
              * goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'lue_vpdscale'].sort_index()
             )
nirvp_pred = np.array([f for f in range(0, 200)])
ndvip_scal_pred = np.array([f for f in range(0, 200)])

lrc_nirvp_fit = ((err_sums.loc['lrc_nirvp' + season, 'alpha'] 
                 * nirvp_pred 
                  * err_sums.loc['lrc_nirvp' + season, 'beta_o'])
                 / (err_sums.loc['lrc_nirvp' + season, 'beta_o']
                    + (err_sums.loc['lrc_nirvp' + season, 'alpha'] 
                       * nirvp_pred)
              )
                )


lin_nirvp_fit = (err_sums.loc['lin_nirvp' + season, 'eps_nirvp'] * nirvp_pred)

lue_fit = (err_sums.loc['lue_ndvip' + season, 'eps_max'] 
           * ndvip_scal_pred
          ) 

#Set up plot
axs[2, 0].set_xlim([-1, max(ndvip_scal) + 3])
axs[2, 0].set_ylim([-1, max(gpp) + 3])

axs[2, 1].set_xlim([-1, max(nirvp) + 3])
axs[2, 1].set_ylim([-1, max(gpp) + 3])

axs[2, 2].set_xlim([-1, max(ppfd) + 3])
axs[2, 2].set_ylim([-1, max(gpp) + 3])

#KDE
z_gpp_ndvip = gaussian_kde(np.vstack([gpp, ndvip_scal]))(np.vstack([gpp, ndvip_scal]))
z_gpp_nirvp = gaussian_kde(np.vstack([gpp, nirvp]))(np.vstack([gpp, nirvp]))
z_gpp_ppfd = gaussian_kde(np.vstack([gpp, ppfd]))(np.vstack([gpp, ppfd]))

im7 = axs[2, 0].scatter (ndvip_scal,
                      gpp,
                      c = z_gpp_ndvip,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[2, 0].plot(ndvip_scal_pred,
                lue_fit,
                color = 'black',
                linewidth = 4
               )


im8 = axs[2, 1].scatter (nirvp,
                      gpp,
                      c = z_gpp_nirvp,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[2, 1].plot(nirvp_pred,
                lin_nirvp_fit,
                color = 'black',
                linewidth = 4,
               linestyle = 'dashed'
               )
axs[2, 1].plot(nirvp_pred,
                lrc_nirvp_fit,
               linewidth = 4,
                color = 'black'
               )


im9 = axs[2, 2].scatter (ppfd,
                      gpp,
                      c = z_gpp_ppfd,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

#------Fall------------------------------------

dat_mask = (goes_tonzi_gs.month >= 10) & (goes_tonzi_gs.month <= 11)
dat_mask = (goes_tonzi_gs.month == 11)
plt_mask = (goes_tonzi_gs[nirv_apar] >= 0) & (goes_tonzi_gs.PPFD_IN_1_1_1 >= 0) #filler mask

season = '11'

#Set data
gpp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
ppfd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'PPFD_IN_1_1_1'].sort_index()
nirvp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , nirv_apar].sort_index()
ndvip_scal = (goes_tonzi_gs.loc[dat_mask].loc[plt_mask , modis_apar].sort_index()
              * goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'lue_tscale'].sort_index()
              * goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'lue_vpdscale'].sort_index()
             )
nirvp_pred = np.array([f for f in range(0, 200)])
ndvip_scal_pred = np.array([f for f in range(0, 200)])

lrc_nirvp_fit = ((err_sums.loc['lrc_nirvp' + season, 'alpha'] 
                 * nirvp_pred 
                  * err_sums.loc['lrc_nirvp' + season, 'beta_o'])
                 / (err_sums.loc['lrc_nirvp' + season, 'beta_o']
                    + (err_sums.loc['lrc_nirvp' + season, 'alpha'] 
                       * nirvp_pred)
              )
                )


lin_nirvp_fit = (err_sums.loc['lin_nirvp' + season, 'eps_nirvp'] * nirvp_pred)

lue_fit = (err_sums.loc['lue_ndvip' + season, 'eps_max'] 
           * ndvip_scal_pred
          ) 

#Set up plot
axs[3, 0].set_xlim([-1, max(ndvip_scal) + 3])
axs[3, 0].set_ylim([-1, max(gpp) + 3])

axs[3, 1].set_xlim([-1, max(nirvp) + 3])
axs[3, 1].set_ylim([-1, max(gpp) + 3])

axs[3, 2].set_xlim([-1, max(ppfd) + 3])
axs[3, 2].set_ylim([-1, max(gpp) + 3])

#KDE
z_gpp_ndvip = gaussian_kde(np.vstack([gpp, ndvip_scal]))(np.vstack([gpp, ndvip_scal]))
z_gpp_nirvp = gaussian_kde(np.vstack([gpp, nirvp]))(np.vstack([gpp, nirvp]))
z_gpp_ppfd = gaussian_kde(np.vstack([gpp, ppfd]))(np.vstack([gpp, ppfd]))

im10 = axs[3, 0].scatter (ndvip_scal,
                      gpp,
                      c = z_gpp_ndvip,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[3, 0].plot(ndvip_scal_pred,
                lue_fit,
                color = 'black',
                   linewidth = 4
               )


im11 = axs[3, 1].scatter (nirvp,
                      gpp,
                      c = z_gpp_nirvp,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[3, 1].plot(nirvp_pred,
                lin_nirvp_fit,
                color = 'black',
                linewidth = 4,
               linestyle = 'dashed'
               )
axs[3, 1].plot(nirvp_pred,
                lrc_nirvp_fit,
                color = 'black',
                linewidth = 4
               )


im12 = axs[3, 2].scatter (ppfd,
                      gpp,
                      c = z_gpp_ppfd,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )


#Labels
axs[0, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 20)
axs[1, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 20)
axs[2, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 20)
axs[3, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 20)


axs[3, 0].set_xlabel(r'$T_{scale}\ \times W_{scale}\ \times$' + 'NDVIP (' + r'$W\ m^{-2})$', fontsize = 20)
axs[3, 1].set_xlabel('NIRvP (' + r'$W\ m^{-2})$', fontsize = 20)
axs[3, 2].set_xlabel('PPFD (' + r'$\mu mol\ Photons\ m^{-2}\ s^{-1})$', fontsize = 20)

#Colorbars
cbar1 = fig.colorbar(im1, ax = axs[0, 0])
cbar1.ax.tick_params(labelsize=18) 
cbar2 = fig.colorbar(im2, ax = axs[0, 1])
cbar2.ax.tick_params(labelsize=18) 
cbar3 = fig.colorbar(im3, ax = axs[0, 2])
cbar3.ax.tick_params(labelsize=18) 

cbar4 = fig.colorbar(im4, ax = axs[1, 0])
cbar4.ax.tick_params(labelsize=18) 
cbar5 = fig.colorbar(im5, ax = axs[1, 1])
cbar5.ax.tick_params(labelsize=18) 
cbar6 = fig.colorbar(im6, ax = axs[1, 2])
cbar6.ax.tick_params(labelsize=18) 

cbar7 = fig.colorbar(im7, ax = axs[2, 0])
cbar7.ax.tick_params(labelsize=18) 
cbar8 = fig.colorbar(im8, ax = axs[2, 1])
cbar8.ax.tick_params(labelsize=18) 
cbar9 = fig.colorbar(im9, ax = axs[2, 2])
cbar9.ax.tick_params(labelsize=18) 

cbar10 = fig.colorbar(im10, ax = axs[3, 0])
cbar10.ax.tick_params(labelsize=18) 
cbar11 = fig.colorbar(im11, ax = axs[3, 1])
cbar11.ax.tick_params(labelsize=18) 
cbar12 = fig.colorbar(im12, ax = axs[3, 2])
cbar12.ax.tick_params(labelsize=18) 

#Titles
axs[0, 1].set_title('Feb', fontsize = 25)
axs[1, 1].set_title('May', fontsize = 25)
axs[2, 1].set_title('Jul', fontsize = 25)
axs[3, 1].set_title('Nov', fontsize = 25)

#Save plot
plt.savefig('light_response_allseas.jpg',
            transparent = True,
            dpi = 300
           )

plt.show()

# Figure 2

In [ ]:
#Set up plot for environmental responses
fig, axs = plt.subplots(4, 3, figsize=(15,19), constrained_layout = True)

#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)

#----------------------Winter--------------------
dat_mask = (goes_tonzi_gs.month >= 1) & (goes_tonzi_gs.month <= 3)
dat_mask = (goes_tonzi_gs.month == 1)
plt_mask = goes_tonzi_gs[gpp_field] > 0 #filler mask

#Set up y-axis for lue stressors
axs[0, 1].set_ylim([0, 1.2])
axs[0, 2].set_ylim([0, 1.2])

#Set data 
vpd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , vpd_field].sort_index()
gpp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
beta_vpd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'beta'].sort_index()
t_scale = goes_tonzi_gs.loc[dat_mask, 'lue_tscale'].sort_index()
t_air = goes_tonzi_gs.loc[dat_mask, t_air_field].sort_index()
vpd_scale = goes_tonzi_gs.loc[dat_mask, 'lue_vpdscale'].sort_index()
vpd_lue = goes_tonzi_gs.loc[dat_mask, vpd_field].sort_index()

#KDE
z_vpd_gpp = gaussian_kde(np.vstack([gpp, vpd]))(np.vstack([gpp, vpd]))

im1 = axs[0, 0].scatter (vpd,
                      gpp,
                      c = z_vpd_gpp,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[0, 0].scatter (vpd,
                beta_vpd,
                color = 'black'
               )

axs[0, 1].scatter (t_air,
                  t_scale,
                  color = 'black'
                 )


axs[0, 2].scatter (vpd_lue,
                  vpd_scale,
                  color = 'black'
                 )

#----------------------Spring--------------------
dat_mask = (goes_tonzi_gs.month >= 4) & (goes_tonzi_gs.month <= 6)
dat_mask = (goes_tonzi_gs.month == 5)
plt_mask = (goes_tonzi_gs['beta'] > 0) #filler mask

#Set up y-axis for lue stressors
axs[1, 1].set_ylim([0, 1.2])
axs[1, 2].set_ylim([0, 1.2])

#Set data 
vpd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , vpd_field].sort_index()
gpp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
beta_vpd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'beta'].sort_index()
t_scale = goes_tonzi_gs.loc[dat_mask, 'lue_tscale'].sort_index()
t_air = goes_tonzi_gs.loc[dat_mask, t_air_field].sort_index()
vpd_scale = goes_tonzi_gs.loc[dat_mask, 'lue_vpdscale'].sort_index()
vpd_lue = goes_tonzi_gs.loc[dat_mask, vpd_field].sort_index()

#KDE
z_vpd_gpp = gaussian_kde(np.vstack([gpp, vpd]))(np.vstack([gpp, vpd]))

im2 = axs[1, 0].scatter (vpd,
                      gpp,
                      c = z_vpd_gpp,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[1, 0].scatter (vpd,
                beta_vpd,
                color = 'black'
               )

axs[1, 1].scatter (t_air,
                  t_scale,
                  color = 'black'
                 )


axs[1, 2].scatter (vpd_lue,
                  vpd_scale,
                  color = 'black'
                 )

#----------------------Summer--------------------
dat_mask = (goes_tonzi_gs.month >= 7) & (goes_tonzi_gs.month <= 9)
dat_mask = (goes_tonzi_gs.month == 9)
plt_mask = (goes_tonzi_gs['beta'] > 0) & (goes_tonzi_gs['beta'] < 100) #filler mask

#Set up y-axis for lue stressors
axs[2, 1].set_ylim([0, 1.2])
axs[2, 2].set_ylim([0, 1.2])

#Set data 
vpd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , vpd_field].sort_index()
gpp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
beta_vpd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'beta'].sort_index()
t_scale = goes_tonzi_gs.loc[dat_mask, 'lue_tscale'].sort_index()
t_air = goes_tonzi_gs.loc[dat_mask, t_air_field].sort_index()
vpd_scale = goes_tonzi_gs.loc[dat_mask, 'lue_vpdscale'].sort_index()
vpd_lue = goes_tonzi_gs.loc[dat_mask, vpd_field].sort_index()

#KDE
z_vpd_gpp = gaussian_kde(np.vstack([gpp, vpd]))(np.vstack([gpp, vpd]))

im3 = axs[2, 0].scatter (vpd,
                      gpp,
                      c = z_vpd_gpp,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[2, 0].scatter (vpd,
                beta_vpd,
                color = 'black'
               )

axs[2, 1].scatter (t_air,
                  t_scale,
                  color = 'black'
                 )


axs[2, 2].scatter (vpd_lue,
                  vpd_scale,
                  color = 'black'
                 )

#----------------------Fall--------------------
dat_mask = (goes_tonzi_gs.month >= 10) & (goes_tonzi_gs.month <= 12)
dat_mask = (goes_tonzi_gs.month == 12)
plt_mask = (goes_tonzi_gs['beta'] > 0) #filler mask

#Set up y-axis for lue stressors
axs[3, 1].set_ylim([0, 1.2])
axs[3, 2].set_ylim([0, 1.2])

#Set data 
vpd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , vpd_field].sort_index()
gpp = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
beta_vpd = goes_tonzi_gs.loc[dat_mask].loc[plt_mask , 'beta'].sort_index()
t_scale = goes_tonzi_gs.loc[dat_mask, 'lue_tscale'].sort_index()
t_air = goes_tonzi_gs.loc[dat_mask, t_air_field].sort_index()
vpd_scale = goes_tonzi_gs.loc[dat_mask, 'lue_vpdscale'].sort_index()
vpd_lue = goes_tonzi_gs.loc[dat_mask, vpd_field].sort_index()

#KDE
z_vpd_gpp = gaussian_kde(np.vstack([gpp, vpd]))(np.vstack([gpp, vpd]))

im4 = axs[3, 0].scatter (vpd,
                      gpp,
                      c = z_vpd_gpp,
                      cmap = plt.get_cmap('viridis'),
                      alpha = 0.6
                     )

axs[3, 0].scatter (vpd,
                beta_vpd,
                color = 'black'
               )

axs[3, 1].scatter (t_air,
                  t_scale,
                  color = 'black'
                 )


axs[3, 2].scatter (vpd_lue,
                  vpd_scale,
                  color = 'black'
                 )


#Labels
axs[0, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 25)
axs[1, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 25)
axs[2, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 25)
axs[3, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 25)

axs[0, 1].set_ylabel('$T_{air}$ stress on ' +  r'$\epsilon_{max}$', fontsize = 25)
axs[1, 1].set_ylabel('$T_{air}$ stress on ' +  r'$\epsilon_{max}$', fontsize = 25)
axs[2, 1].set_ylabel('$T_{air}$ stress on ' +  r'$\epsilon_{max}$', fontsize = 25)
axs[3, 1].set_ylabel('$T_{air}$ stress on ' +  r'$\epsilon_{max}$', fontsize = 25)

axs[0, 2].set_ylabel('VPD stress on ' +  r'$\epsilon_{max}$', fontsize = 25)
axs[1, 2].set_ylabel('VPD stress on ' +  r'$\epsilon_{max}$', fontsize = 25)
axs[2, 2].set_ylabel('VPD stress on ' +  r'$\epsilon_{max}$', fontsize = 25)
axs[3, 2].set_ylabel('VPD stress on ' +  r'$\epsilon_{max}$', fontsize = 25)

axs[3, 0].set_xlabel('VPD (hPa)', fontsize = 25)
axs[3, 1].set_xlabel('$T_{air}$ (C)', fontsize = 25)
axs[3, 2].set_xlabel('VPD (hPa)', fontsize = 25)

#Colorbars
cbar1 = fig.colorbar(im1, ax = axs[0, 0])
cbar1.ax.tick_params(labelsize=18) 
cbar2 = fig.colorbar(im2, ax = axs[1, 0])
cbar2.ax.tick_params(labelsize=18) 
cbar3 = fig.colorbar(im3, ax = axs[2, 0])
cbar3.ax.tick_params(labelsize=18) 
cbar4 = fig.colorbar(im4, ax = axs[3, 0])
cbar4.ax.tick_params(labelsize=18)

#Titles
axs[0, 1].set_title('Jan', fontsize = 25)
axs[1, 1].set_title('May', fontsize = 25)
axs[2, 1].set_title('Sep', fontsize = 25)
axs[3, 1].set_title('Dec', fontsize = 25)

#Save plot
# plt.savefig('env_stress_allseas.jpg',
#             transparent = True,
#             dpi = 300
#            )

plt.show()

# Figure 3

In [ ]:
#Add month in tonzi_orig
tonzi_orig.loc[:, 'month'] = tonzi_orig.index.month

In [ ]:
#Plot timeseries
fig, axs = plt.subplots(5, figsize=(20,30), constrained_layout=True)

#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)

mask = (goes_tonzi_gs[gpp_field] > 0)
orig_mask = (tonzi_orig[gpp_field] < 125) 

##############################################------------Winter
start_date = '2020-02-12'
end_date = '2020-02-19'

axs[0].plot(tonzi_orig.loc[orig_mask].sort_index().loc[start_date: end_date].index,
            tonzi_orig.loc[orig_mask].sort_index().loc[start_date: end_date][gpp_field],
            label = 'Tower',
            zorder = 0,
            color = 'purple',
            linewidth = 2
           )

axs[0].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lue_ndvip'],
            label = 'LUE-NDVI',
            marker = 's',
            s = 80,
            facecolors = 'none',
            edgecolor = 'mediumblue')

axs[0].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lin_nirvp'],
            label = 'LIN-NIRvP',
            marker = 'v',
            s = 80,
            facecolors = 'none',
            edgecolor = 'black')

axs[0].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lrc_nirvp'],
            label = 'LRC-NIRvP',
            marker = 'o',
            s = 80,
            facecolors = 'none',
            edgecolor = 'teal')


axs[0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 25)

##############################################------------Spring
start_date = '2020-05-20'
end_date = '2020-05-28'

axs[1].plot(tonzi_orig.loc[orig_mask].sort_index().loc[start_date: end_date].index,
            tonzi_orig.loc[orig_mask].sort_index().loc[start_date: end_date][gpp_field],
            label = 'Tower',
            zorder = 0,
            color = 'purple',
            linewidth = 2
           )

axs[1].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lue_ndvip'],
            label = 'LUE-NDVI',
            marker = 's',
            s = 80,
            facecolors = 'none',
            edgecolor = 'mediumblue')

axs[1].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lin_nirvp'],
            label = 'LIN-NIRvP',
            marker = 'v',
            s = 80,
            facecolors = 'none',
            edgecolor = 'black')

axs[1].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lrc_nirvp'],
            label = 'LRC-NIRvP',
            marker = 'o',
            s = 80,
            facecolors = 'none',
            edgecolor = 'teal')


##############################################------------Summer
start_date = '2020-07-01'
end_date = '2020-07-08'

axs[2].plot(tonzi_orig.loc[orig_mask].sort_index().loc[start_date: end_date].index,
            tonzi_orig.loc[orig_mask].sort_index().loc[start_date: end_date][gpp_field],
            label = 'Tower',
            zorder = 0,
            color = 'purple',
            linewidth = 2
           )

axs[2].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lue_ndvip'],
            label = 'LUE-NDVI',
            marker = 's',
            s = 80,
            facecolors = 'none',
            edgecolor = 'mediumblue')

axs[2].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lin_nirvp'],
            label = 'LIN-NIRvP',
            marker = 'v',
            s = 80,
            facecolors = 'none',
            edgecolor = 'black')

axs[2].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lrc_nirvp'],
            label = 'LRC-NIRvP',
            marker = 'o',
            s = 80,
            facecolors = 'none',
            edgecolor = 'teal')


##############################################------------Fall
start_date = '2019-11-01'
end_date = '2019-11-08'


axs[3].plot(tonzi_orig.loc[orig_mask].sort_index().loc[start_date: end_date].index,
            tonzi_orig.loc[orig_mask].sort_index().loc[start_date: end_date][gpp_field],
            label = 'Tower',
            zorder = 0,
            color = 'purple',
            linewidth = 2
           )

axs[3].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lue_ndvip'],
            label = 'LUE-NDVI',
            marker = 's',
            s = 80,
            facecolors = 'none',
            edgecolor = 'mediumblue')

axs[3].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lin_nirvp'],
            label = 'LIN-NIRvP',
            marker = 'v',
            s = 80,
            facecolors = 'none',
            edgecolor = 'black')

axs[3].scatter(goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask].index,
            goes_tonzi_gs.sort_index().loc[start_date: end_date].loc[mask]['lrc_nirvp'],
            label = 'LRC-NIRvP',
            marker = 'o',
            s = 80,
            facecolors = 'none',
            edgecolor = 'teal')

##############################################------------Full year
start_date = '2019-01-01'
end_date = '2020-12-31'

axs[4].plot(goes_tonzi_gs.loc[start_date: end_date].loc[mask].resample('8D').mean().index,
            goes_tonzi_gs.loc[start_date: end_date].loc[mask].resample('8D').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
            linewidth = 3
           )

axs[4].plot(goes_tonzi_gs.loc[start_date: end_date].loc[mask].resample('8D').mean().index,
            goes_tonzi_gs.loc[start_date: end_date].loc[mask].resample('8D').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
            linewidth = 3,
            linestyle = '--'
           )

axs[4].plot(goes_tonzi_gs.loc[start_date: end_date].loc[mask].resample('8D').mean().index,
            goes_tonzi_gs.loc[start_date: end_date].loc[mask].resample('8D').mean()['lin_nirvp'],
            label = 'LIN-NIRvP',
            color = 'black',
            linewidth = 3
           )
axs[4].plot(goes_tonzi_gs.loc[start_date: end_date].loc[mask].resample('8D').mean().index,
            goes_tonzi_gs.loc[start_date: end_date].loc[mask].resample('8D').mean()['lrc_nirvp'],
            label = 'LRC-NIRvP',
            color = 'teal',
            linewidth = 3
           )

#Turn on grid
axs[0].grid()
axs[1].grid()
axs[2].grid()
axs[3].grid()
axs[4].grid()

#Legend
axs[3].legend(loc = 'center', fontsize = 25, bbox_to_anchor=(0.4, -0.3), ncol = 5)
axs[4].legend(loc = 'center', fontsize = 25, bbox_to_anchor=(0.4, -0.3), ncol = 5)

#Axis labels
axs[0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
axs[1].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
axs[2].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
axs[3].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
axs[4].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)


#Save plot
plt.savefig('allmodels_gpp_timeseries_seasonal_nirvref.jpg',
            transparent = True,
            dpi = 300
           )


plt.show()

# Figure 4

In [ ]:
#Plot timeseries of means by hour
fig, axs = plt.subplots(2, 2, figsize=(21, 18), constrained_layout= False)


#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)

##############################################------------Winter
mask = (goes_tonzi_gs[gpp_field] > 0)
plot_date_mask = ((goes_tonzi_gs.month <= 3) & (goes_tonzi_gs.month >= 1))


axs[0, 0].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
         linewidth = 8
           )

#Add tower gpp std err
axs[0, 0].fill_between(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
                       (goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        + ( 2 * goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       (goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        - ( 2 * goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       color = 'purple',
                       alpha = 0.1
           )

#Add other estimates
axs[0, 0].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
         linewidth = 5,
               linestyle = '--'
           )


axs[0, 0].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lin_nirvp'],
         label = 'LIN-NIRvP',
         color = 'black',
         linewidth = 5
           )

axs[0, 0].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_nirvp'],
         label = 'LRC-NIRvP',
         color = 'teal',
         linewidth = 5
           )

##############################################------------Spring
mask = (goes_tonzi_gs[gpp_field] > 0)
plot_date_mask = ((goes_tonzi_gs.month <= 6) & (goes_tonzi_gs.month >= 4))

axs[0, 1].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
         linewidth = 8
           )
#Add tower gpp std err
axs[0, 1].fill_between(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
                       (goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        + ( 2 * goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       (goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        - ( 2 * goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       color = 'purple',
                       alpha = 0.1
           )

#Add other estimates
axs[0, 1].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
         linewidth = 5,
               linestyle = '--'
           )


axs[0, 1].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lin_nirvp'],
         label = 'LIN-NIRvP',
         color = 'black',
         linewidth = 5
           )

axs[0, 1].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_nirvp'],
         label = 'LRC-NIRvP',
         color = 'teal',
         linewidth = 5
           )

##############################################------------Summer
plot_date_mask = ((goes_tonzi_gs.month <= 9) & (goes_tonzi_gs.month >= 7))


axs[1, 0].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
         linewidth = 8
           )
#Add tower gpp std err
axs[1, 0].fill_between(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
                       (goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        + ( 2 * goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       (goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        - ( 2 * goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       color = 'purple',
                       alpha = 0.1
           )

#Add other estimates
axs[1, 0].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
         linewidth = 5,
               linestyle = '--'
           )


axs[1, 0].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lin_nirvp'],
         label = 'LIN-NIRvP',
         color = 'black',
         linewidth = 5
           )

axs[1, 0].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_nirvp'],
         label = 'LRC-NIRvP',
         color = 'teal',
         linewidth = 5
           )

##############################################------------Fall
plot_date_mask = ((goes_tonzi_gs.month <= 12) & (goes_tonzi_gs.month >= 10))

axs[1, 1].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
         linewidth = 8
           )
#Add tower gpp std err
axs[1, 1].fill_between(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
                       (goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        + ( 2 * goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       (goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        - ( 2 * goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       color = 'purple',
                       alpha = 0.1
           )


#Add other estimates
axs[1, 1].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
         linewidth = 5,
               linestyle = '--'
           )


axs[1, 1].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lin_nirvp'],
         label = 'LIN-NIRvP',
         color = 'black',
         linewidth = 5
           )

axs[1, 1].plot(goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_nirvp'],
         label = 'LRC-NIRvP',
         color = 'teal',
         linewidth = 5
           )


#Axis labels
axs[0, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 35)
axs[1, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 35)
axs[1, 1].set_xlabel('Hour', fontsize = 35)
axs[1, 0].set_xlabel('Hour', fontsize = 35)

#Legends
axs[1, 1].legend(loc = 'center', fontsize = 35, bbox_to_anchor=(-0.1, -0.25), ncol = 5)

#Grids
axs[0, 0].grid()
axs[0, 1].grid()
axs[1, 0].grid()
axs[1, 1].grid()

#Titles
axs[0, 0].set_title('Winter: Jan - Mar', fontsize = 35)
axs[0, 1].set_title('Spring: Apr - Jun', fontsize = 35)
axs[1, 0].set_title('Summer: Jul - Sep', fontsize = 35)
axs[1, 1].set_title('Fall: Oct - Dec', fontsize = 35)

#Save plot
plt.savefig('gpp_halfhourly_means_seasonal.jpg',
            transparent = True,
            dpi = 300
           )


plt.show()

# Figure 5

In [ ]:
#Plot training data and save plot

#Set up plot
fig, axs = plt.subplots(3, 4, figsize=(27,18), constrained_layout = False)

#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)

#Make a list of models to plot
models = ['lue_ndvip', 'lin_nirvp', 'lrc_nirvp']

#Set counter 
x = 0
y = 0

for model in models:
    #-----------------------------------Diurnal data------------------------------
    #-------Robust linear model for diurnal data
    #Training data
    gpp_mask_train = (goes_tonzi_gs.train_oth == '1') & (~goes_tonzi_gs[model].isna())

    goes_gpp_train = goes_tonzi_gs.loc[gpp_mask_train, model]
    tower_gpp_train = goes_tonzi_gs.loc[gpp_mask_train, gpp_field]
    tower_gpp_const_train = sm.add_constant(tower_gpp_train)

    #Model and predict
    gpp_train_oth_RLM = sm.RLM(goes_gpp_train, tower_gpp_const_train).fit()
    gpp_preds_train = gpp_train_oth_RLM.predict(tower_gpp_const_train)

    #Test data
    gpp_mask_test = (goes_tonzi_gs.test_oth == '1') & (~goes_tonzi_gs[model].isna())

    goes_gpp_test = goes_tonzi_gs.loc[gpp_mask_test, model]
    tower_gpp_test = goes_tonzi_gs.loc[gpp_mask_test, gpp_field]
    tower_gpp_const_test = sm.add_constant(tower_gpp_test)

    #Model and predict
    gpp_test_oth_RLM = sm.RLM(goes_gpp_test, tower_gpp_const_test).fit()
    gpp_preds_test = gpp_test_oth_RLM.predict(tower_gpp_const_test)
    
    #------Plot diurnal validation
    #---Test data
    max_x = max(max(goes_gpp_test), tower_gpp_test.max()) + 3
    max_y = max(max(goes_gpp_test), tower_gpp_test.max()) + 3
    axs[x, y].set_xlim([-0.2,
              max_x])
    axs[x, y].set_ylim([-0.2,
              max_y])

    #KDE
    z_gpp = gaussian_kde(np.vstack([tower_gpp_test, goes_gpp_test]))(np.vstack([tower_gpp_test, goes_gpp_test]))

    im1 = axs[x, y].scatter(tower_gpp_test.sort_index(), 
                goes_gpp_test.sort_index(),
                #color = 'gray',
                c = z_gpp,
                cmap = plt.get_cmap('inferno')
               )

    #Add 1:1 line
    axs[x, y].plot(np.arange(-1.,max_x + 5, 1),
             np.arange(-1.,max_y + 5, 1), 
             color = 'black',
             linewidth = 3
            )

    #Add regression line
    axs[x, y].plot(tower_gpp_test.sort_index(),
             gpp_test_oth_RLM.fittedvalues.sort_index(), 
             color = 'gray',
             linewidth = 5
            )

    #Add equations
    text = r'$\^y$' + " = " + str(gpp_test_oth_RLM.params[0].round(2)) + ' + ' + str(gpp_test_oth_RLM.params[1].round(2)) + 'x'
    axs[x, y].text(1, 
             max_y - 3, 
             text,
             fontsize = 30)

    #----Train data
    max_x = max(max(goes_gpp_train), tower_gpp_train.max()) + 2
    max_y = max(max(goes_gpp_train), tower_gpp_train.max()) + 2
    axs[x, y + 1].set_xlim([-0.2,
              max_x])
    axs[x, y + 1].set_ylim([-0.2,
              max_y])

    #KDE
    z_gpp_train = gaussian_kde(np.vstack([tower_gpp_train, goes_gpp_train]))(np.vstack([tower_gpp_train, goes_gpp_train]))

    im2 = axs[x, y + 1].scatter(tower_gpp_train.sort_index(), 
                goes_gpp_train.sort_index(),
                #alpha = 0.5,
                c = z_gpp_train,
                cmap = plt.get_cmap('inferno')
               )

    #Add 1:1 line
    axs[x, y + 1].plot(np.arange(-1.,max_x + 5, 1),
             np.arange(-1.,max_y + 5, 1), 
             color = 'black',
             linewidth = 3
            )

    #Add regression line
    axs[x, y + 1].plot(tower_gpp_train.sort_index(),
             gpp_train_oth_RLM.fittedvalues.sort_index(), 
             color = 'gray',
             linewidth = 5
            )

    #Add equations
    text = r'$\^y$' + " = " + str(gpp_train_oth_RLM.params[0].round(2)) + ' + ' + str(gpp_train_oth_RLM.params[1].round(2)) + 'x'
    axs[x, y + 1].text(1, 
             max_y - 3, 
             text,
             fontsize = 30)
    
    #-----------------------------------Daily data------------------------------
    #-------Robust linear model for daily data
    #Training data
    gpp_mask_train = (goes_tonzi_gs.train_oth == '1')

    goes_gpp_train = goes_tonzi_gs.loc[gpp_mask_train].groupby('date').median()[model]
    tower_gpp_train = goes_tonzi_gs.loc[gpp_mask_train].groupby('date').median()[gpp_field]
    tower_gpp_const_train = sm.add_constant(tower_gpp_train)

    #Model and predict
    gpp_train_oth_RLM = sm.RLM(goes_gpp_train, tower_gpp_const_train).fit()
    gpp_preds_train = gpp_train_oth_RLM.predict(tower_gpp_const_train)

    #Test data
    gpp_mask_test = (goes_tonzi_gs.test_oth == '1')

    goes_gpp_test = goes_tonzi_gs.loc[gpp_mask_test].groupby('date').median()[model]
    tower_gpp_test = goes_tonzi_gs.loc[gpp_mask_test].groupby('date').median()[gpp_field]
    tower_gpp_const_test = sm.add_constant(tower_gpp_test)

    #Model and predict
    gpp_test_oth_RLM = sm.RLM(goes_gpp_test, tower_gpp_const_test).fit()
    gpp_preds_test = gpp_test_oth_RLM.predict(tower_gpp_const_test)
    
    #------Plot daily validation
    #---Test data
    max_x = max(max(goes_gpp_test), tower_gpp_test.max()) + 3
    max_y = max(max(goes_gpp_test), tower_gpp_test.max()) + 3
    axs[x, y + 2].set_xlim([-0.2,
              max_x])
    axs[x, y + 2].set_ylim([-0.2,
              max_y])

    #KDE
    z_gpp = gaussian_kde(np.vstack([tower_gpp_test, goes_gpp_test]))(np.vstack([tower_gpp_test, goes_gpp_test]))

    im1 = axs[x, y + 2].scatter(tower_gpp_test.sort_index(), 
                goes_gpp_test.sort_index(),
                #color = 'gray',
                c = z_gpp,
                cmap = plt.get_cmap('inferno')
               )

    #Add 1:1 line
    axs[x, y + 2].plot(np.arange(-1.,max_x + 5, 1),
             np.arange(-1.,max_y + 5, 1), 
             color = 'black',
             linewidth = 3
            )

    #Add regression line
    axs[x, y + 2].plot(tower_gpp_test.sort_index(),
             gpp_test_oth_RLM.fittedvalues.sort_index(), 
             color = 'gray',
             linewidth = 5
            )

    #Add equations
    text = r'$\^y$' + " = " + str(gpp_test_oth_RLM.params[0].round(2)) + ' + ' + str(gpp_test_oth_RLM.params[1].round(2)) + 'x'
    axs[x, y + 2].text(1, 
             max_y - 3, 
             text,
             fontsize = 30)

    #----Train data
    max_x = max(max(goes_gpp_train), tower_gpp_train.max()) + 2
    max_y = max(max(goes_gpp_train), tower_gpp_train.max()) + 2
    axs[x, y + 3].set_xlim([-0.2,
              max_x])
    axs[x, y + 3].set_ylim([-0.2,
              max_y])

    #KDE
    z_gpp_train = gaussian_kde(np.vstack([tower_gpp_train, goes_gpp_train]))(np.vstack([tower_gpp_train, goes_gpp_train]))

    im2 = axs[x, y + 3].scatter(tower_gpp_train.sort_index(), 
                goes_gpp_train.sort_index(),
                #alpha = 0.5,
                c = z_gpp_train,
                cmap = plt.get_cmap('inferno')
               )

    #Add 1:1 line
    axs[x, y + 3].plot(np.arange(-1.,max_x + 5, 1),
             np.arange(-1.,max_y + 5, 1), 
             color = 'black',
             linewidth = 3
            )

    #Add regression line
    axs[x, y + 3].plot(tower_gpp_train.sort_index(),
             gpp_train_oth_RLM.fittedvalues.sort_index(), 
             color = 'gray',
             linewidth = 5
            )

    #Add equations
    text = r'$\^y$' + " = " + str(gpp_train_oth_RLM.params[0].round(2)) + ' + ' + str(gpp_train_oth_RLM.params[1].round(2)) + 'x'
    axs[x, y + 3].text(1, 
             max_y - 3, 
             text,
             fontsize = 30)
    
#     #Axis titles
#     axs[x, y].set_ylabel('Estimated GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)

#     if x == 2:
#         axs[x, y].set_xlabel('Tower GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
#         axs[x, y + 1].set_xlabel('Tower GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
        
    #Color bars
    cbar1 = fig.colorbar(im1, ax = axs[x, y])
    cbar1.ax.tick_params(labelsize=18) 
    cbar2 = fig.colorbar(im2, ax = axs[x, y + 1])
    cbar2.ax.tick_params(labelsize=18) 
    cbar3 = fig.colorbar(im2, ax = axs[x, y + 2])
    cbar3.ax.tick_params(labelsize=18) 
    cbar4 = fig.colorbar(im2, ax = axs[x, y + 3])
    cbar4.ax.tick_params(labelsize=18) 
    
    #titles
    if model == 'lue_ndvip':
        axs[x, y].set_title ('A. Diurnal Test: LUE-NDVI', fontsize = 25)
        axs[x, y + 1].set_title ('B. Diurnal Train: LUE-NDVI', fontsize = 25)
        axs[x, y + 2].set_title ('C. Daily Test: LUE-NDVI', fontsize = 25)
        axs[x, y + 3].set_title ('D. Daily Train: LUE-NDVI', fontsize = 25)
        
    if model == 'lin_nirvp':
        axs[x, y].set_title ('E. Diurnal Test: LIN-NIRvP', fontsize = 25)
        axs[x, y + 1].set_title ('F. Diurnal Train: LIN-NIRvP', fontsize = 25)
        axs[x, y + 2].set_title ('G. Daily Test: LIN-NIRvP', fontsize = 25)
        axs[x, y + 3].set_title ('H. Daily Train: LIN-NIRvP', fontsize = 25)
        
    if model == 'lrc_nirvp':
        axs[x, y].set_title ('I. Diurnal Test: LRC-NIRvP', fontsize = 25)
        axs[x, y + 1].set_title ('J. Diurnal Train: LRC-NIRvP', fontsize = 25)
        axs[x, y + 2].set_title ('K. Daily Test: LRC-NIRvP', fontsize = 25)
        axs[x, y + 3].set_title ('L. Daily Train: LRC-NIRvP', fontsize = 25)
        
    #Update counter
    x = x + 1

#Axis labels
plt.xlabel('', 
         fontsize = 35)
plt.ylabel('', 
         fontsize = 35)
fig.text(0.09, 0.5, 
         'Estimated GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', 
         fontsize = 35, 
         ha = 'center', 
         va = 'center',
         rotation = 'vertical'
        )

fig.text(0.5, 0.08, 
         'Tower GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', 
         fontsize = 35, 
         ha = 'center', 
         va = 'center'
        )

#Save plot
plt.savefig('test_train_gpp_scatter.jpg',
            transparent = True,
            bbox_inches = 'tight',
            dpi = 300
           )


plt.show()

# Figure 6

In [ ]:
goes_tonzi_gs.loc[:, 'lue_ndvip_err'] = (goes_tonzi_gs.loc[:, 'lue_ndvip']
                                                         - goes_tonzi_gs.loc[:, gpp_field]
                                                        )

goes_tonzi_gs.loc[:, 'lin_nirvp_err'] = (goes_tonzi_gs.loc[:, 'lin_nirvp']
                                                         - goes_tonzi_gs.loc[:, gpp_field]
                                                        )
goes_tonzi_gs.loc[:, 'lrc_nirvp_err'] = (goes_tonzi_gs.loc[:, 'lrc_nirvp']
                                                         - goes_tonzi_gs.loc[:, gpp_field]
                                                        )

In [ ]:
#Plot training data and save plot

#Set up plot
fig, axs = plt.subplots(3, 2, figsize=(17, 20), constrained_layout = True)

#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)

plt_mask = (goes_tonzi_gs['apar_nirv_rdd_0vza_ppfd'] >= 0) & (goes_tonzi_gs['apar_nirv_ref_tower'] >= 0) & (goes_tonzi_gs['PPFD_IN_1_1_1'] >= 0)
mask_oth = (goes_tonzi_gs.test_oth == '1') 

#Set data
lue_err = goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'lue_ndvip_err']

goes_ndvi_ppfd_lue = (goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'apar_ndvi_rdd_0vza_ppfd']
                   - goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'PPFD_IN_1_1_1'])

tower_ndvi_ppfd_lue = (goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'apar_ndvi_ref_tower']
                    - goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'PPFD_IN_1_1_1'])

lin_nirvp_err = goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'lin_nirvp_err']

lrc_nirvp_err = goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'lrc_nirvp_err']

tower_nirvp_ppfd = (goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'apar_nirv_ref_tower']
                    - goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'PPFD_IN_1_1_1'])

goes_nirvp_ppfd = (goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'apar_nirv_rdd_0vza_ppfd']
                   - goes_tonzi_gs.loc[plt_mask].loc[mask_oth, 'PPFD_IN_1_1_1'])


#KDE
z_goes_ndvi_lue = gaussian_kde(np.vstack([goes_ndvi_ppfd_lue,
                                           lue_err]))(np.vstack([goes_ndvi_ppfd_lue,
                                                                 lue_err]))
z_tower_ndvi_lue = gaussian_kde(np.vstack([tower_ndvi_ppfd_lue, 
                                            lue_err]))(np.vstack([tower_ndvi_ppfd_lue, 
                                                                  lue_err]))

im1 = axs[0, 0].scatter(goes_ndvi_ppfd_lue,
                        lue_err,
                        c = z_goes_ndvi_lue,
                        cmap = plt.get_cmap('inferno')
                       )

im2 = axs[0, 1].scatter(tower_ndvi_ppfd_lue,
                        lue_err,
                        c = z_tower_ndvi_lue,
                        cmap = plt.get_cmap('inferno')
                       )
#---------------------

#KDE
z_goes_nirvp_lin = gaussian_kde(np.vstack([goes_nirvp_ppfd,
                                           lin_nirvp_err]))(np.vstack([goes_nirvp_ppfd,
                                                                 lin_nirvp_err]))
z_tower_nirvp_lin = gaussian_kde(np.vstack([tower_nirvp_ppfd, 
                                            lin_nirvp_err]))(np.vstack([tower_nirvp_ppfd, 
                                                                  lin_nirvp_err]))

im3 = axs[1, 0].scatter(goes_nirvp_ppfd,
                        lin_nirvp_err,
                        c = z_goes_nirvp_lin,
                        cmap = plt.get_cmap('inferno')
                       )

im4 = axs[1, 1].scatter(tower_nirvp_ppfd,
                        lin_nirvp_err,
                        c = z_tower_nirvp_lin,
                        cmap = plt.get_cmap('inferno')
                       )
#---------------------
#KDE
z_goes_nirvp_lrc_nirvp = gaussian_kde(np.vstack([goes_nirvp_ppfd,
                                           lrc_nirvp_err]))(np.vstack([goes_nirvp_ppfd,
                                                                 lrc_nirvp_err]))
z_tower_nirvp_lrc_nirvp = gaussian_kde(np.vstack([tower_nirvp_ppfd, 
                                            lrc_nirvp_err]))(np.vstack([tower_nirvp_ppfd, 
                                                                  lrc_nirvp_err]))

im5 = axs[2, 0].scatter(goes_nirvp_ppfd,
                        lrc_nirvp_err,
                        c = z_goes_nirvp_lrc_nirvp,
                        cmap = plt.get_cmap('inferno')
                       )

im6 = axs[2, 1].scatter(tower_nirvp_ppfd,
                        lrc_nirvp_err,
                        c = z_tower_nirvp_lrc_nirvp,
                        cmap = plt.get_cmap('inferno')
                       )

#Add colorbars
cbar1 = fig.colorbar(im1, ax = axs[0, 0])
cbar1.ax.tick_params(labelsize=18) 
cbar2 = fig.colorbar(im2, ax = axs[0, 1])
cbar2.ax.tick_params(labelsize=18) 

cbar3 = fig.colorbar(im3, ax = axs[1, 0])
cbar3.ax.tick_params(labelsize=18) 
cbar4 = fig.colorbar(im4, ax = axs[1, 1])
cbar4.ax.tick_params(labelsize=18)

cbar5 = fig.colorbar(im5, ax = axs[2, 0])
cbar5.ax.tick_params(labelsize=18) 
cbar6 = fig.colorbar(im6, ax = axs[2, 1])
cbar6.ax.tick_params(labelsize=18)

#Add 0 lines
axs[0, 0].axhline(0, color = 'black', linewidth = 3)
axs[0, 1].axhline(0, color = 'black', linewidth = 3)

axs[1, 0].axhline(0, color = 'black', linewidth = 3)
axs[1, 1].axhline(0, color = 'black', linewidth = 3)

axs[2, 0].axhline(0, color = 'black', linewidth = 3)
axs[2, 1].axhline(0, color = 'black', linewidth = 3)


#Axis labels
axs[0, 0].set_ylabel('LUE-NDVI GPP - Tower GPP', fontsize = 25)
axs[0, 0].set_xlabel(r'$ABI\ NDVIP_{PPFD} - Tower\ PPFD$', fontsize = 25)
axs[0, 1].set_xlabel(r'$Tower\ NDVIP_{PPFD} - Tower\ PPFD$', fontsize = 25)

axs[1, 0].set_ylabel('LIN-NIRvP GPP - Tower GPP', fontsize = 25)
axs[2, 0].set_ylabel('LRC-NIRvP GPP - Tower GPP', fontsize = 25)

axs[2, 0].set_xlabel(r'$ABI\ NIRvP_{PPFD} - Tower\ PPFD$', fontsize = 25)
axs[2, 1].set_xlabel(r'$Tower\ NIRvP_{PPFD} - Tower\ PPFD$', fontsize = 25)

#Axis titles
axs[0, 0].set_title(r'$\bf{A}$', loc = 'left', fontsize = 30)
axs[1, 0].set_title(r'$\bf{B}$', loc = 'left', fontsize = 30)
axs[2, 0].set_title(r'$\bf{C}$', loc = 'left', fontsize = 30)

#Save plot
plt.savefig('gpp_errs_nirvp_ppfd.jpg',
            transparent = True,
            dpi = 300
           )

plt.show()

# Diurnal centroids

In [ ]:
#Calculate diurnal centroid: equation 22
for day in goes_tonzi_gs.date:
    #Set mask for date and hours
    dmask = (goes_tonzi_gs.date == day) & (goes_tonzi_gs.dec_hour >= 9) & (goes_tonzi_gs.dec_hour <= 15) & (goes_tonzi_gs.SW_IN_1_1_1 >= 0)
    
    #Calculate sum of fluxes by day and add the sum column to goes_tonzi_gs
    goes_tonzi_gs.loc[dmask, 'dtot_le'] = goes_tonzi_gs.loc[dmask].groupby('date')[le_field].transform('sum')
    goes_tonzi_gs.loc[dmask, 'dtot_vpd'] = goes_tonzi_gs.loc[dmask].groupby('date')[vpd_field].transform('sum')
    goes_tonzi_gs.loc[dmask, 'dtot_sw'] = goes_tonzi_gs.loc[dmask].groupby('date')['SW_IN_1_1_1'].transform('sum')
    goes_tonzi_gs.loc[dmask, 'dtot_gpp_tower'] = goes_tonzi_gs.loc[dmask].groupby('date')[gpp_field].transform('sum')
    goes_tonzi_gs.loc[dmask, 'dtot_lin_nirvp'] = goes_tonzi_gs.loc[dmask].groupby('date')['lin_nirvp'].transform('sum')
    goes_tonzi_gs.loc[dmask, 'dtot_lrc_nirvp'] = goes_tonzi_gs.loc[dmask].groupby('date')['lrc_nirvp'].transform('sum')
    goes_tonzi_gs.loc[dmask, 'dtot_lue_ndvip'] = goes_tonzi_gs.loc[dmask].groupby('date')['lue_ndvip'].transform('sum')
    
    #Calculate centroid
    goes_tonzi_gs.sort_index(inplace = True)
    goes_tonzi_gs.loc[dmask, 'dcent_le'] = (np.inner(goes_tonzi_gs.loc[dmask, le_field], 
                                                     goes_tonzi_gs.loc[dmask, 'dec_hour']) 
                                            / goes_tonzi_gs.loc[dmask, 'dtot_le'])
    goes_tonzi_gs.loc[dmask, 'dcent_vpd'] = (np.inner(goes_tonzi_gs.loc[dmask, vpd_field],
                                                      goes_tonzi_gs.loc[dmask, 'dec_hour']) 
                                            / goes_tonzi_gs.loc[dmask, 'dtot_vpd'])
    goes_tonzi_gs.loc[dmask, 'dcent_sw'] = (np.inner(goes_tonzi_gs.loc[dmask, 'SW_IN_1_1_1'], 
                                                     goes_tonzi_gs.loc[dmask, 'dec_hour'])
                                           / goes_tonzi_gs.loc[dmask, 'dtot_sw'])
    goes_tonzi_gs.loc[dmask, 'dcent_gpp_tower'] = (np.inner(goes_tonzi_gs.loc[dmask, gpp_field], 
                                                            goes_tonzi_gs.loc[dmask, 'dec_hour'])
                                                  / goes_tonzi_gs.loc[dmask, 'dtot_gpp_tower'])
    goes_tonzi_gs.loc[dmask, 'dcent_lin_nirvp'] = (np.inner(goes_tonzi_gs.loc[dmask, 'lin_nirvp'], 
                                                                goes_tonzi_gs.loc[dmask, 'dec_hour'])
                                                     / goes_tonzi_gs.loc[dmask, 'dtot_lin_nirvp'])
    goes_tonzi_gs.loc[dmask, 'dcent_lrc_nirvp'] = (np.inner(goes_tonzi_gs.loc[dmask, 'lrc_nirvp'], 
                                                                goes_tonzi_gs.loc[dmask, 'dec_hour'])
                                                         / goes_tonzi_gs.loc[dmask, 'dtot_lrc_nirvp'])
    goes_tonzi_gs.loc[dmask, 'dcent_lue_ndvip'] = (np.inner(goes_tonzi_gs.loc[dmask, 'lue_ndvip'], 
                                                                goes_tonzi_gs.loc[dmask, 'dec_hour'])
                                                      /goes_tonzi_gs.loc[dmask, 'dtot_lue_ndvip'])

In [ ]:
#Check calculations
hr_mask = (goes_tonzi_gs.dec_hour >= 9) & (goes_tonzi_gs.dec_hour <= 15)
goes_tonzi_gs.loc['2019-08-10'].loc[hr_mask][[gpp_field, 'dec_hour', 'dtot_gpp_tower', 'dcent_gpp_tower', 'lrc_nirvp', 'dtot_lrc_nirvp', 'dcent_lrc_nirvp']]

In [ ]:
#This table will give us daily diurnal centroids by date
goes_tonzi_dcents = goes_tonzi_gs.loc[~goes_tonzi_gs.dcent_lin_nirvp.isna()].groupby('date').mean()

#Add month name and season
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 1., 'month_name'] = 'Jan'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 2., 'month_name'] = 'Feb'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 3., 'month_name'] = 'Mar'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 4., 'month_name'] = 'Apr'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 5., 'month_name'] = 'May'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 6., 'month_name'] = 'Jun'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 7., 'month_name'] = 'Jul'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 8., 'month_name'] = 'Aug'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 9., 'month_name'] = 'Sep'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 10., 'month_name'] = 'Oct'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 11., 'month_name'] = 'Nov'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 12., 'month_name'] = 'Dec'


goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 1., 'season'] = 'Winter'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 2., 'season'] = 'Winter'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 3., 'season'] = 'Winter'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 4., 'season'] = 'Spring'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 5., 'season'] = 'Spring'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 6., 'season'] = 'Spring'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 7., 'season'] = 'Summer'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 8., 'season'] = 'Summer'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 9., 'season'] = 'Summer'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 10., 'season'] = 'Fall'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 11., 'season'] = 'Fall'
goes_tonzi_dcents.loc[goes_tonzi_dcents.month == 12., 'season'] = 'Fall'

#Get differences in daily diurnal centroids
#eq. 24
goes_tonzi_dcents.loc[:, 'dcent_gpp_tower_le'] = (goes_tonzi_dcents.loc[:, 'dcent_gpp_tower']
                                                  - goes_tonzi_dcents.loc[:, 'dcent_le'])
#eq. 22
goes_tonzi_dcents.loc[:, 'dcent_gpp_tower_sw'] = (goes_tonzi_dcents.loc[:, 'dcent_gpp_tower']
                                                  - goes_tonzi_dcents.loc[:, 'dcent_sw'])
goes_tonzi_dcents.loc[:, 'dcent_le_sw'] = (goes_tonzi_dcents.loc[:, 'dcent_le']
                                           - goes_tonzi_dcents.loc[:, 'dcent_sw'])
goes_tonzi_dcents.loc[:, 'dcent_vpd_sw'] = (goes_tonzi_dcents.loc[:, 'dcent_vpd']
                                            - goes_tonzi_dcents.loc[:, 'dcent_sw'])


#eq. 24
goes_tonzi_dcents.loc[:, 'dcent_lue_ndvip_le'] = (goes_tonzi_dcents.loc[:, 'dcent_lue_ndvip']
                                                  - goes_tonzi_dcents.loc[:, 'dcent_le'])
#eq. 22
goes_tonzi_dcents.loc[:, 'dcent_lue_ndvip_sw'] = (goes_tonzi_dcents.loc[:, 'dcent_lue_ndvip']
                                                  - goes_tonzi_dcents.loc[:, 'dcent_sw'])

#eq. 24
goes_tonzi_dcents.loc[:, 'dcent_lin_nirvp_le'] = (goes_tonzi_dcents.loc[:, 'dcent_lin_nirvp']
                                                  - goes_tonzi_dcents.loc[:, 'dcent_le'])
#eq. 22
goes_tonzi_dcents.loc[:, 'dcent_lin_nirvp_sw'] = (goes_tonzi_dcents.loc[:, 'dcent_lin_nirvp']
                                                  - goes_tonzi_dcents.loc[:, 'dcent_sw'])

#eq. 24
goes_tonzi_dcents.loc[:, 'dcent_lrc_nirvp_le'] = (goes_tonzi_dcents.loc[:, 'dcent_lrc_nirvp']
                                                  - goes_tonzi_dcents.loc[:, 'dcent_le'])
#eq. 22
goes_tonzi_dcents.loc[:, 'dcent_lrc_nirvp_sw'] = (goes_tonzi_dcents.loc[:, 'dcent_lrc_nirvp']
                                                  - goes_tonzi_dcents.loc[:, 'dcent_sw'])

#Get differences between tower diurnal centroid of GPP and diurnal centroids of other GPP estimates
goes_tonzi_dcents.loc[:, 'dcent_tower_lue_gpp'] = (goes_tonzi_dcents.loc[:, 'dcent_lue_ndvip']
                                                   - goes_tonzi_dcents.loc[:, 'dcent_gpp_tower'])
goes_tonzi_dcents.loc[:, 'dcent_tower_lin_gpp'] = (goes_tonzi_dcents.loc[:, 'dcent_lin_nirvp']
                                                   - goes_tonzi_dcents.loc[:, 'dcent_gpp_tower'])
goes_tonzi_dcents.loc[:, 'dcent_tower_lrc_gpp'] = (goes_tonzi_dcents.loc[:, 'dcent_lrc_nirvp']
                                                   - goes_tonzi_dcents.loc[:, 'dcent_gpp_tower'])

In [ ]:
#Make data for plotting boxplots...drop duplicate columns
dcents_plot = goes_tonzi_dcents.melt(id_vars = ['month_name'], value_vars = ['SWC_PI_F_1_1_A',
                                                                             'dcent_gpp_tower',
                                                                             'dcent_le',
                                                                             'dcent_vpd',
                                                                             'dcent_gpp_tower_sw',
                                                                             'dcent_le_sw',
                                                                             'dcent_vpd_sw',
                                                                             'dcent_gpp_tower_le',
                                                                             'dcent_lrc_nirvp_le',
                                                                             'dcent_lrc_nirvp_sw',
                                                                             'dcent_lue_ndvip_le',
                                                                             'dcent_lue_ndvip_sw',
                                                                             'dcent_lin_nirvp_le',
                                                                             'dcent_lin_nirvp_sw',
                                                                             'dcent_tower_lrc_gpp',
                                                                             'dcent_tower_lue_gpp',
                                                                             'dcent_tower_lin_gpp'
                                                                            ]
                                    )

dcents_plot

In [ ]:
#fig, ax1 = plt.subplots(figsize=(25,10))
fig, axs = plt.subplots(4, figsize=(25,30), constrained_layout=True)

#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 35)
plt.rc('ytick', labelsize = 35)

sb.set_style ("white")
#sb.set(font_scale = 1.4)

my_pal = {"dcent_gpp_tower_sw": "purple", 
          "dcent_lrc_nirvp_sw": "teal", 
          "dcent_lue_ndvip_sw":"mediumblue",
          "dcent_lin_nirvp_sw": "gray"
         }

sb.boxplot(x = 'month_name', 
           y = 'value', 
           showfliers = False,
           hue = 'variable', 
           ax = axs[0],
           palette = my_pal,
           data = dcents_plot[dcents_plot.variable.isin(['dcent_gpp_tower_sw',
                                                         'dcent_lrc_nirvp_sw',
                                                         'dcent_lue_ndvip_sw',
                                                         'dcent_lin_nirvp_sw'
                                                        ])])

#Add sample sizes
nobs = dcents_plot[dcents_plot.variable.isin(['dcent_gpp_tower_sw',
                                              'dcent_lrc_nirvp_sw',
                                              'dcent_lue_ndvip_sw',
                                              'dcent_lin_nirvp_sw'
                                             ])].groupby(['variable', 'month_name'], sort = False).agg(['count'])
pos = range(len(nobs))
for tick,label in zip(pos,axs[0].get_xticklabels()):
    axs[0].text(pos[tick], -1.0, 
                'n = ' + str(nobs.reset_index().loc[tick].value[0]), 
                horizontalalignment ='center', 
                fontsize=35, 
                color='black', 
                weight='semibold')

my_pal = {"dcent_gpp_tower_le": "purple", 
          "dcent_lrc_nirvp_le": "teal", 
          "dcent_lue_ndvip_le":"mediumblue",
          "dcent_lin_nirvp_le": "gray"
         }
sb.boxplot(x = 'month_name', 
           y = 'value', 
           showfliers = False,
           hue = 'variable', 
           ax = axs[1],
           palette = my_pal,
           data = dcents_plot[dcents_plot.variable.isin(['dcent_gpp_tower_le',
                                                         'dcent_lrc_nirvp_le',
                                                         'dcent_lue_ndvip_le',
                                                         'dcent_lin_nirvp_le'
                                                        ])])

my_pal = {"dcent_gpp_tower_sw": "purple", 
          "dcent_le_sw": "gold", 
          "dcent_vpd_sw":"blue"
         }
sb.boxplot(x = 'month_name', 
           y = 'value', 
           showfliers = False,
           hue = 'variable', 
           ax = axs[2],
           palette = my_pal,
           data = dcents_plot[dcents_plot.variable.isin(['dcent_gpp_tower_sw',
                                                         'dcent_le_sw',
                                                         'dcent_vpd_sw'
                                                        ])])

#Add soil moisture
my_pal = {"SWC_PI_F_1_1_A": "maroon"}
sb.boxplot(x = 'month_name', 
           y = 'value', 
           showfliers = False,
           hue = 'variable', 
           ax = axs[3],
           palette = my_pal,
           data = dcents_plot[dcents_plot.variable.isin(['SWC_PI_F_1_1_A'
                                                        ])])

#Add 0 lines
axs[0].axhline(y=0, color='r', linestyle='-')
axs[1].axhline(y=0, color='r', linestyle='-')
axs[2].axhline(y=0, color='r', linestyle='-')
#axs[3].axhline(y=0, color='r', linestyle='-')

#Add axis labels
axs[0].set_ylabel('$C_{GPP} - C_{SW_{in}}$', fontsize = 35)
axs[0].set_xlabel('', fontsize = 30)
axs[1].set_ylabel('$C_{GPP} - C_{LE}$', fontsize = 30)
axs[1].set_xlabel('', fontsize = 30)
axs[2].set_ylabel('$C_{var} - C_{SW_{in}}$', fontsize = 35)
axs[2].set_xlabel('', fontsize = 20)
axs[3].set_ylabel('Volumetric Soil Water Content (%)', fontsize = 35)
axs[3].set_xlabel('Month', fontsize = 35)

#Legends
h0, l = axs[0].get_legend_handles_labels()
axs[0].legend(h0, ["Tower", "LRC-VPD", "LUE-NDVI", "LIN-NIRvP"], fontsize = 30, ncol = 5)
h1, l = axs[1].get_legend_handles_labels()
axs[1].legend(h1, ["Tower", "LRC-VPD", "LUE-NDVI", "LIN-NIRvP"], fontsize = 30, ncol = 5)
h2, l = axs[2].get_legend_handles_labels()
axs[2].legend(h2, ["Tower GPP", "Tower LE", "Tower VPD"], fontsize = 30, ncol = 3)
axs[3].get_legend().remove()

#Grids
axs[0].grid()
axs[1].grid()
axs[2].grid()
axs[3].grid()

#LAbel with letters
axs[0].set_title(r'$\bf{A}$', loc = 'left', fontsize = 40)
axs[1].set_title(r'$\bf{B}$', loc = 'left', fontsize = 40)
axs[2].set_title(r'$\bf{C}$', loc = 'left', fontsize = 40)
axs[3].set_title(r'$\bf{D}$', loc = 'left', fontsize = 40)

plt.savefig('allmodels_dcent_compare_nirvref.jpg',
            transparent = True,
            dpi = 300
           )

plt.show()

In [ ]:
#Check all legends are labeled properly
axs[0].get_legend_handles_labels(), axs[1].get_legend_handles_labels(), axs[2].get_legend_handles_labels(), axs[3].get_legend_handles_labels()

In [ ]:
np.unique(dcents_plot.variable)

# Appendix A

# Read in LST data

In [ ]:
#Read in GOES LST values at neon/tonzi sites
goes_lst = pd.read_csv(r'path to file')

#Get GOES LST for Tonzi
goes_lst = goes_lst.loc[(goes_lst.siteCode == 'US-Ton') & (goes_lst.LST_DQF == 0.0)]

#Drop dubplicate dates for LST using stimestamp column
goes_lst.drop_duplicates(subset = 'stimestamp', inplace = True)

# Fix timestamps

In [ ]:
#Convert goes lst to tonzi time and set as index
goes_lst.loc [:,'stimestamp'] = pd.to_datetime (goes_lst.loc [:, 'stimestamp'])
goes_lst.loc[:, 'stimestamp_utcminus8'] = goes_lst.loc[:, 'stimestamp']
goes_lst = goes_lst.set_index('stimestamp_utcminus8')
goes_lst.index = goes_lst.index.tz_convert(offset)

In [ ]:
offset

# Interpolate hourly LST and merge data

In [ ]:
#Get good quality and interpolate hourly LST
goes_lst = goes_lst.loc[goes_lst.LST_DQF == 0.0].sort_index()
goes_lst.index = goes_lst.index.floor ('H') #Take floor of timestamp so resampling intervals are consistent

#Upsample LST to half hour
lst_30 = pd.DataFrame(goes_lst['LST']).sort_index().resample('30T', 
                                                             closed = 'left', 
                                                             label = 'left').interpolate('linear',
                                                                                         limit_area = 'inside')

#Add time and date
lst_30.loc[:, 'lst_date'] = lst_30.index.date
lst_30.loc[:, 'lst_time'] = lst_30.index.time

In [ ]:
goes_tonzi_lst = pd.merge_asof(goes_tonzi_gs.sort_index(),
                               lst_30[['LST',
                                       'lst_date',
                                       'lst_time']].sort_index(),
                               left_index = True,
                               right_index = True,
                               allow_exact_matches = True,
                               direction = 'forward',
                               suffixes = ('', '_lst'),
                               tolerance = pd.Timedelta("0 days 00:15:00")
                              )

In [ ]:
#Check join
goes_tonzi_lst.loc[~goes_tonzi_lst.lst_time.isna()].sort_index()[['LST', 'lst_date', 'lst_time']]

# Table A1

In [ ]:
#Equation A1 and A2
def find_lrc_lst (params, x_dat, return_beta = False):
    x_dat.sort_index(inplace = True)
    #eq. A2
    x_dat.loc[:, 'beta'] = params[1] / (1 + np.exp( (-params[2] * (x_dat.loc[:, 'LST'] - params[3])).astype('float128')))

    if return_beta:
        return(x_dat['beta'])
    
    #eq. A1
    return ((params[0] * x_dat.loc[:, nirv_apar] * x_dat.loc[:, 'beta']) 
            / (x_dat.loc[:, 'beta'] + (params[0] * x_dat.loc[:, nirv_apar])))

def resid_lrc_lst (params, x_dat, y_dat):
    return (find_lrc_lst(params, x_dat) - y_dat)

In [ ]:
seasons = ['winter', 'spring', 'summer', 'fall']

colnames = ['rmse_train', 'nrmse_train', 'me_train', 'mae_train', 'nmae_train',
            'rmse_test', 'nrmse_test', 'me_test', 'mae_test', 'nmae_test']

#Make empty dataframe which will be populated below
err_sums = pd.DataFrame()

#Make empty list which will be populated by tables generated below
month_tables = []

#test
#season = 'spring'

for season in seasons:
    
    #-----Split data into test and train sets--------------------------
    #Make season
    if season == 'winter':
        date_mask = ((goes_tonzi_lst.month <= 3) & (goes_tonzi_lst.month >= 1))
        mask2 = (goes_tonzi_lst.LST > 280) | (goes_tonzi_lst[gpp_field] < 10) #take out unusually high GPP during winter (based on scatter plot)
    if season == 'spring':
        date_mask = ((goes_tonzi_lst.month <= 6) & (goes_tonzi_lst.month >= 4))
        mask2 = (goes_tonzi_lst.LST > 280)
    if season == 'summer':
        date_mask = ((goes_tonzi_lst.month <= 9) & (goes_tonzi_lst.month >= 7))
        mask2 = (goes_tonzi_lst.LST > 290) 
    if season == 'fall':
        date_mask = ((goes_tonzi_lst.month <= 12) & (goes_tonzi_lst.month >= 10))
        mask2 = (goes_tonzi_lst.LST > 270)

    #Make mask for valid observations
    mask = (goes_tonzi_lst[gpp_field] > 0) & (goes_tonzi_lst[gpp_field] < 25) & (goes_tonzi_lst.P < 10)

    #Make date mask according to month
    if season == 'full_year':
        goes_tonzi_lst_gs = goes_tonzi_lst.loc[mask].loc[mask2].sort_index()

    else:
        #Make growing season data
        goes_tonzi_lst_gs = goes_tonzi_lst.loc[date_mask].loc[mask].loc[mask2].sort_index()
        goes_tonzi_lst_gs.sort_index(inplace = True)
        
    #Split data
    oth_mask = (goes_tonzi_lst_gs[nirv_apar] >= 0) 

    x_train, x_test, y_train, y_test = train_test_split(goes_tonzi_lst_gs.loc[oth_mask].sort_index(), 
                                                                        goes_tonzi_lst_gs.loc[oth_mask].sort_index(), 
                                                                        test_size = 0.30,
                                                                        random_state = 10
                                                                       )

    #----------Estimate parameters---------------------------------------------
    #Set initial values for parameters
    
    #LRC-LST 
    gpp_high_quant_mask = x_train[gpp_field] >= x_train[gpp_field].quantile(q=0.75)
    gpp_low_quant_mask = x_train[gpp_field] <= x_train[gpp_field].quantile(q=0.25)
    lst_o = x_train.loc[gpp_high_quant_mask, 'LST'].mean()

    grt_mask = x_train['LST'] > lst_o
    less_mask = x_train['LST'] <= lst_o
    
    alpha_o = (x_train.loc[gpp_low_quant_mask, gpp_field]
               / x_train.loc[gpp_low_quant_mask, nirv_apar]).mean()
    beta_o = x_train[gpp_field].max() 
    
    b = (x_train.loc[grt_mask, gpp_field] 
         / x_train.loc[grt_mask, 'LST']).median()  
    b = 0.3

    
    print ('b = ' + str(b), 'lst_o = ' + str(lst_o))


    #Call least_sqaures with residuals
    #Set bounds: ([lower], [upper])
    lrc_bounds = ([0, 0, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf])
    model_lrc_lst = optimize.least_squares(resid_lrc_lst,
                                           x0 = [alpha_o, beta_o, b, lst_o],
                                           args = (x_train,
                                                   y_train[gpp_field]),
                                           bounds = lrc_bounds,
                                           loss = 'huber'
                                          )
    print('lrc-lst done')

    #-------------Get error summaries----------------------------------------
    #Get training and testing estimated and tower GPP
    train_pred_gpp = find_lrc_lst (params = model_lrc_lst.x,
                                   x_dat = x_train)
    test_pred_gpp = find_lrc_lst (params = model_lrc_lst.x,
                                  x_dat = x_test)

    #Add 'lrc_lst' and season to error summaries table
    err_sums.loc['lrc_lst' + season, 'season'] = season
    err_sums.loc['lrc_lst' + season, 'model'] = 'lrc_lst'

    #Add 'lrc_lst' parameters and initial values to error summaries table
    err_sums.loc['lrc_lst' + season, 'alpha'] = model_lrc_lst.x[0]
    err_sums.loc['lrc_lst' + season, 'beta_o'] = model_lrc_lst.x[1]
    err_sums.loc['lrc_lst' + season, 'b'] = model_lrc_lst.x[2]
    err_sums.loc['lrc_lst' + season, 'lst_o'] = model_lrc_lst.x[3]

    err_sums.loc['lrc_lst' + season, 'beta_o_o'] = beta_o
    err_sums.loc['lrc_lst' + season, 'b_o'] = b
    err_sums.loc['lrc_lst' + season, 'lst_o_o'] = lst_o
    err_sums.loc['lrc_lst' + season, 'alpha_o'] = alpha_o
            
    #Set train and test tower gpp
    train_tower_gpp = y_train[gpp_field]
    test_tower_gpp = y_test[gpp_field]
        
        
    #----Calculate RMSE, NRMSE, ME, MAE, NMAE
    errors_train = train_pred_gpp - train_tower_gpp
    me_train = errors_train.mean()
    rmse_train = np.sqrt((errors_train ** 2).mean())
    nrmse_train = rmse_train / train_tower_gpp.mean()
    mae_train = abs(errors_train).mean()
    nmae_train = mae_train / train_tower_gpp.mean()

    errors_test = test_pred_gpp - test_tower_gpp
    me_test = errors_test.mean()
    rmse_test = np.sqrt((errors_test ** 2).mean())
    nrmse_test = rmse_test / test_tower_gpp.mean()
    mae_test = abs(errors_test).mean()
    nmae_test = mae_test / test_tower_gpp.mean()

    mets = {'rmse_train':rmse_train,
             'nrmse_train': nrmse_train,
             'me_train':me_train,
             'mae_train': mae_train,
             'nmae_train': nmae_train,
             'rmse_test':rmse_test,
             'nrmse_test': nrmse_test,
             'me_test': me_test,
             'mae_test': mae_test,
             'nmae_test': nmae_test
            }

    #Add error summaries to table
    for col in colnames:
        err_sums.loc['lrc_lst' + season, col] = mets[col]

    print ('Parameters estimated')
    
    
    #------Make seasonal table of estimates----------------------------------------------------- 
    oth_mask_gs = (goes_tonzi_lst_gs[nirv_apar] >= 0)

    #add beta
    goes_tonzi_lst_gs.loc[oth_mask_gs, 'beta_lrc_lst'] = find_lrc_lst (params = model_lrc_lst.x,
                                                                   x_dat = goes_tonzi_lst_gs.loc[oth_mask_gs].copy(),
                                                                   return_beta = True)

    goes_tonzi_lst_gs.loc[oth_mask_gs, 'lrc_gpp_lst'] =  find_lrc_lst (params = model_lrc_lst.x,
                                                                   x_dat = goes_tonzi_lst_gs.loc[oth_mask_gs].copy())

    
    #-----------------Mark training and test data-----------
    goes_tonzi_lst_gs.loc[goes_tonzi_lst_gs.index.isin(x_train.index), 'train_lst'] = '1'
    goes_tonzi_lst_gs.loc[goes_tonzi_lst_gs.index.isin(x_test.index), 'test_lst'] = '1'

    goes_tonzi_lst_gs.loc[goes_tonzi_lst_gs.index.isin(x_train.index), 'train_lst_n'] =  len(goes_tonzi_lst_gs.loc[goes_tonzi_lst_gs.train_oth == '1'])
    goes_tonzi_lst_gs.loc[goes_tonzi_lst_gs.index.isin(x_test.index), 'test_lst_n'] = len(goes_tonzi_lst_gs.loc[goes_tonzi_lst_gs.test_oth == '1'])
    
    #Add to month_tables
    month_tables.append(goes_tonzi_lst_gs)

In [ ]:
#err_sums

In [ ]:
#Export error table
err_sums.to_csv('LST_seas_err_sums.csv', 
                header = True, 
                index = False)

In [ ]:
goes_tonzi_lst_gs = pd.concat(month_tables)
goes_tonzi_lst_gs.sort_index(inplace = True)

# Figure A1

In [ ]:
#Set up plot for environmental responses
fig, axs = plt.subplots(2, 2, figsize=(12,10), constrained_layout = True)

#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 20)
plt.rc('ytick', labelsize = 20)

#------Winter------------------
dat_mask = (goes_tonzi_lst_gs.month >= 1) & (goes_tonzi_lst_gs.month <= 3)
season = 'winter'
plt_mask = goes_tonzi_lst_gs[vpd_field] >= 0 #filler mask

#Set data 
lst = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , 'LST'].sort_index()
gpp = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
beta_lst = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , 'beta_lrc_lst'].sort_index()


#KDE
z_lst_gpp = gaussian_kde(np.vstack([gpp, lst]))(np.vstack([gpp, lst]))

im1 = axs[0,0].scatter (lst,
                  gpp,
                  c = z_lst_gpp,
                  cmap = plt.get_cmap('viridis'),
                  alpha = 0.6
              )

axs[0,0].scatter (lst,
                  beta_lst,
                  color = 'black'
              )

#------Spring------------------
dat_mask = (goes_tonzi_lst_gs.month >= 4) & (goes_tonzi_lst_gs.month <= 6)
season = 'spring'
plt_mask = goes_tonzi_lst_gs[vpd_field] >= 0 #filler mask

#Set data 
lst = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , 'LST'].sort_index()
gpp = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
beta_lst = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , 'beta_lrc_lst'].sort_index()


#KDE
z_lst_gpp = gaussian_kde(np.vstack([gpp, lst]))(np.vstack([gpp, lst]))

im2 = axs[0,1].scatter (lst,
                  gpp,
                  c = z_lst_gpp,
                  cmap = plt.get_cmap('viridis'),
                  alpha = 0.6
              )

axs[0,1].scatter (lst,
                  beta_lst,
                  color = 'black'
              )

#------Summer------------------
dat_mask = (goes_tonzi_lst_gs.month >= 7) & (goes_tonzi_lst_gs.month <= 9)
season = 'summer'
plt_mask = goes_tonzi_lst_gs[vpd_field] >= 0 #filler mask

#Set data 
lst = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , 'LST'].sort_index()
gpp = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
beta_lst = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , 'beta_lrc_lst'].sort_index()

#KDE
z_lst_gpp = gaussian_kde(np.vstack([gpp, lst]))(np.vstack([gpp, lst]))

im3 = axs[1,0].scatter (lst,
                  gpp,
                  c = z_lst_gpp,
                  cmap = plt.get_cmap('viridis'),
                  alpha = 0.6
              )

axs[1,0].scatter (lst,
                  beta_lst,
                  color = 'black'
              )

#------Fall------------------
dat_mask = (goes_tonzi_lst_gs.month >= 10) & (goes_tonzi_lst_gs.month <= 12)
season = 'fall'
plt_mask = goes_tonzi_lst_gs[vpd_field] >= 0 #filler mask

#Set data 
lst = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , 'LST'].sort_index()
gpp = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , gpp_field].sort_index()
beta_lst = goes_tonzi_lst_gs.loc[dat_mask].loc[plt_mask , 'beta_lrc_lst'].sort_index()


#KDE
z_lst_gpp = gaussian_kde(np.vstack([gpp, lst]))(np.vstack([gpp, lst]))

im4 = axs[1,1].scatter (lst,
                  gpp,
                  c = z_lst_gpp,
                  cmap = plt.get_cmap('viridis'),
                  alpha = 0.6
              )

axs[1,1].scatter (lst,
                  beta_lst,
                  color = 'black'
              )

#Labels
axs[0, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 20)
axs[1, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 20)

axs[1, 0].set_xlabel('LST(K)', fontsize = 20)
axs[1, 1].set_xlabel('LST (K)', fontsize = 20)

#Colorbars
cbar1 = fig.colorbar(im1, ax = axs[0, 0])
cbar1.ax.tick_params(labelsize=18) 
cbar2 = fig.colorbar(im2, ax = axs[0, 1])
cbar2.ax.tick_params(labelsize=18) 
cbar3 = fig.colorbar(im3, ax = axs[1, 0])
cbar3.ax.tick_params(labelsize=18) 
cbar4 = fig.colorbar(im4, ax = axs[1, 1])
cbar4.ax.tick_params(labelsize=18) 


axs[0, 0].set_title('Winter: Jan - Mar', fontsize = 30)
axs[0, 1].set_title('Spring: Apr - Jun', fontsize = 30)
axs[1, 0].set_title('Summer: Jul - Sep', fontsize = 30)
axs[1, 1].set_title('Fall: Oct - Dec', fontsize = 30)

#Save plot
plt.savefig('LST_env_response.jpg',
            transparent = True,
            dpi = 300
           )

plt.show()

# Figure A2

In [ ]:
#Plot timeseries of medians by hour
fig, axs = plt.subplots(2, 2, figsize=(21, 18), constrained_layout= False)


#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)

mask = (goes_tonzi_lst_gs[gpp_field] > 0)

#####---------------------Winter
plot_date_mask = ((goes_tonzi_lst_gs.month <= 3) & (goes_tonzi_lst_gs.month >= 1))


axs[0, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
         linewidth = 8
           )

#Add std err
axs[0, 0].fill_between(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
                       (goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        + (2 * goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       (goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        - (2 * goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       color = 'purple',
                       alpha = 0.1
           )

#Add other estimates
axs[0, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
         linewidth = 5,
               linestyle = '--'
           )


axs[0, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lin_nirvp'],
         label = 'LIN-NIRvP',
         color = 'black',
         linewidth = 5
           )

axs[0, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_nirvp'],
         label = 'LRC-NIRvP',
         color = 'teal',
         linewidth = 5
           )


axs[0, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_gpp_lst'],
         label = 'LRC-NIRvP-LST',
         color = 'red',
         linewidth = 5
           )

#####---------------------Spring
plot_date_mask = ((goes_tonzi_lst_gs.month <= 6) & (goes_tonzi_lst_gs.month >= 4))

axs[0, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
         linewidth = 8
           )
#Add std err
axs[0, 1].fill_between(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
                       (goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        + (2 * goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       (goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        - (2 * goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       color = 'purple',
                       alpha = 0.1
           )

#Add other estimates
axs[0, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
         linewidth = 5,
               linestyle = '--'
           )


axs[0, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lin_nirvp'],
         label = 'LIN-NIRvP',
         color = 'black',
         linewidth = 5
           )

axs[0, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_nirvp'],
         label = 'LRC-NIRvP',
         color = 'teal',
         linewidth = 5
           )


axs[0, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_gpp_lst'],
         label = 'LRC-NIRvP-LST',
         color = 'red',
         linewidth = 5
           )

#####---------------------Summer
plot_date_mask = ((goes_tonzi_lst_gs.month <= 9) & (goes_tonzi_lst_gs.month >= 7))

axs[1, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
         linewidth = 8
           )
#Add std err
axs[1, 0].fill_between(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
                       (goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        + (2 * goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       (goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        - (2 * goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       color = 'purple',
                       alpha = 0.1
           )

#Add other estimates
axs[1, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
         linewidth = 5,
               linestyle = '--'
           )


axs[1, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lin_nirvp'],
         label = 'LIN-NIRvP',
         color = 'black',
         linewidth = 5
           )

axs[1, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_nirvp'],
         label = 'LRC-NIRvP',
         color = 'teal',
         linewidth = 5
           )


axs[1, 0].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_gpp_lst'],
         label = 'LRC-NIRvP-LST',
         color = 'red',
         linewidth = 5
           )

#####---------------------Fall
plot_date_mask = ((goes_tonzi_lst_gs.month <= 12) & (goes_tonzi_lst_gs.month >= 10))

axs[1, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field],
            label = 'Tower GPP',
            color = 'purple',
         linewidth = 8
           )

#Add std err
axs[1, 1].fill_between(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
                       (goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        + (2 * goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       (goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()[gpp_field] 
                        - (2 * goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').sem()[gpp_field])),
                       color = 'purple',
                       alpha = 0.1
           )


#Add other estimates
axs[1, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
            goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lue_ndvip'],
            label = 'LUE-NDVI',
            color = 'mediumblue',
         linewidth = 5,
               linestyle = '--'
           )


axs[1, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lin_nirvp'],
         label = 'LIN-NIRvP',
         color = 'black',
         linewidth = 5
           )

axs[1, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_nirvp'],
         label = 'LRC-NIRvP',
         color = 'teal',
         linewidth = 5
           )


axs[1, 1].plot(goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean().index,
         goes_tonzi_lst_gs.loc[mask].loc[plot_date_mask].sort_index().groupby('dec_hour').mean()['lrc_gpp_lst'],
         label = 'LRC-NIRvP-LST',
         color = 'red',
         linewidth = 5
           )

#Axis labels
axs[0, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 35)
axs[1, 0].set_ylabel('GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 35)
axs[1, 0].set_xlabel('Hour', fontsize = 35)
axs[1, 1].set_xlabel('Hour', fontsize = 35)


#Legends
axs[1, 1].legend(loc = 'center', fontsize = 35, bbox_to_anchor=(-0.1, -0.25), ncol = 5)

#Grids
axs[0, 0].grid()
axs[0, 1].grid()
axs[1, 0].grid()
axs[1, 1].grid()

#Titles
axs[0, 0].set_title('Winter: Jan - Mar', fontsize = 35)
axs[0, 1].set_title('Spring: Apr - Jun', fontsize = 35)
axs[1, 0].set_title('Summer: Jul - Sep', fontsize = 35)
axs[1, 1].set_title('Fall: Oct - Dec', fontsize = 35)

#Save plot
plt.savefig('LST_gpp_halfhourly_seasonal.jpg',
            transparent = True,
            bbox_inches = 'tight',
            dpi = 300
           )

plt.show()

In [ ]:
#Validation RLM between GOES GPP and Tower GPP
model = 'lrc_gpp_lst'

#----Training data
gpp_mask_train = (goes_tonzi_lst_gs.train_lst == '1')

goes_gpp_train = goes_tonzi_lst_gs.loc[gpp_mask_train, model]
tower_gpp_train = goes_tonzi_lst_gs.loc[gpp_mask_train, gpp_field]
tower_gpp_const_train = sm.add_constant(tower_gpp_train)

#Model and predict
gpp_train_oth_RLM = sm.RLM(goes_gpp_train, tower_gpp_const_train).fit()
gpp_preds_train = gpp_train_oth_RLM.predict(tower_gpp_const_train)

print(gpp_train_oth_RLM.summary())

#------Test data
gpp_mask_test = (goes_tonzi_lst_gs.test_lst == '1')

goes_gpp_test = goes_tonzi_lst_gs.loc[gpp_mask_test, model]
tower_gpp_test = goes_tonzi_lst_gs.loc[gpp_mask_test, gpp_field]
tower_gpp_const_test = sm.add_constant(tower_gpp_test)

#Model and predict
gpp_test_oth_RLM = sm.RLM(goes_gpp_test, tower_gpp_const_test).fit()
gpp_preds_test = gpp_test_oth_RLM.predict(tower_gpp_const_test)

#gpp_test_oth_RLM.summary()

# Figure A3

In [ ]:
#Plot training data and save plot

#Set up plot
fig, axs = plt.subplots(1, 2, figsize=(17,8), constrained_layout = True)

#Set xtick and ytick label fontsize
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)

max_x = max(max(goes_gpp_test), tower_gpp_test.max()) + 3
max_y = max(max(goes_gpp_test), tower_gpp_test.max()) + 3
axs[0].set_xlim([-0.2,
          max_x])
axs[0].set_ylim([-0.2,
          max_y])

#KDE
z_gpp = gaussian_kde(np.vstack([tower_gpp_test, goes_gpp_test]))(np.vstack([tower_gpp_test, goes_gpp_test]))

im1 = axs[0].scatter(tower_gpp_test.sort_index(), 
            goes_gpp_test.sort_index(),
            c = z_gpp,
            cmap = plt.get_cmap('inferno')
           )

#Add 1:1 line
axs[0].plot(np.arange(-1.,max_x + 5, 1),
         np.arange(-1.,max_y + 5, 1), 
         color = 'black',
         linewidth = 3
        )

#Add regression line
axs[0].plot(tower_gpp_test.sort_index(),
         gpp_test_oth_RLM.fittedvalues.sort_index(), 
         color = 'gray',
         linewidth = 5
        )

#Add equations
text = r'$\^y$' + " = " + str(gpp_test_oth_RLM.params[0].round(2)) + ' + ' + str(gpp_test_oth_RLM.params[1].round(2)) + 'x'
axs[0].text(1, 
         max_y - 3, 
         text,
         fontsize = 30)

axs[0].set_ylabel('Estimated GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
axs[0].set_xlabel('Tower GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)

#----Train
max_x = max(max(goes_gpp_train), tower_gpp_train.max()) + 2
max_y = max(max(goes_gpp_train), tower_gpp_train.max()) + 2
axs[1].set_xlim([-0.2,
          max_x])
axs[1].set_ylim([-0.2,
          max_y])

#KDE
z_gpp_train = gaussian_kde(np.vstack([tower_gpp_train, goes_gpp_train]))(np.vstack([tower_gpp_train, goes_gpp_train]))

im2 = axs[1].scatter(tower_gpp_train.sort_index(), 
            goes_gpp_train.sort_index(),
            #alpha = 0.5,
            c = z_gpp_train,
            cmap = plt.get_cmap('inferno')
           )

#Add 1:1 line
axs[1].plot(np.arange(-1.,max_x + 5, 1),
         np.arange(-1.,max_y + 5, 1), 
         color = 'black',
         linewidth = 3
        )

#Add regression line
axs[1].plot(tower_gpp_train.sort_index(),
         gpp_train_oth_RLM.fittedvalues.sort_index(), 
         color = 'gray',
         linewidth = 5
        )

#Add equations
text = r'$\^y$' + " = " + str(gpp_train_oth_RLM.params[0].round(2)) + ' + ' + str(gpp_train_oth_RLM.params[1].round(2)) + 'x'
axs[1].text(1, 
         max_y - 3, 
         text,
         fontsize = 30)

axs[1].set_ylabel('Estimated GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
axs[1].set_xlabel('Tower GPP (' + r'$\mu mol\ CO_{2}\ m^{-2}\ s^{-1})$', fontsize = 30)
        
cbar1 = fig.colorbar(im1, ax = axs[0])
cbar1.ax.tick_params(labelsize=18) 
cbar2 = fig.colorbar(im2, ax = axs[1])
cbar2.ax.tick_params(labelsize=18) 

#Save plot
plt.savefig('lrc_lst_test_train_gpp_scatter.jpg',
            transparent = True,
            dpi = 300
           )

plt.show()